# Models

<a id='contents'></a>
## Contents

* [Introduction](#introduction)
* [Choice of classifier and metric](#choice_classifier)
* [Data](#data)
* [Setup and loading data](#setup_and)
* [Filtering and train/test split](#filtering)
* [Creating an instance of the model class](#creating_an)
* [A first model (XGBoost)](#first_model)
* [Adjusting classification thresholds](#adjusting_)
* [Simple oversampling](#simple_oversampling)
* [Augmented oversampling](#augmented_oversampling)
* [Combining threshold adjustments and oversampling](#combining)
* [Final model](#final_model)
* [Evaluation](#evaluation)
* [Other classifiers](#other_classifiers)
* [Retrospective analysis](#analysis)
* [Conclusion](#conclusion)

<a id='introduction'></a>
## Introduction
↑↑ [Contents](#contents) ↓ [Choice of classifier and metric](#choice_classifier)

We wish to use up to 17 categorical and three ordinal features of ~281,000 road collision records to classify them as either: 
- 0 (material damage only); 
- 1 (at least one minor injury); 
- 2 (at least one person seriously or fatally injured).

See our [exploratory data analysis](./01-data-exploration-saaq.ipynb) for precise definitions and elucidation of the data. A key takeway from the EDA is that the dataset is highly imbalanced: less than 1% of accidents are in the most severe class, and less than 20% in the middling severity class. Another takeaway is that there are relationships among variables that are difficult to disentangle (we discussed the example of ```SPD_LIM```, ```ZONE```, ```MONTH```, and ```HOUR```).

<a id='choice_classifier'></a>
## Choice of classifier and metric
↑↑ [Contents](#contents) ↑ [Introduction](#introduction) ↓ [Data](#data)

**Classifier.** Considering the complexity of our classification problem, the imbalanced nature of our data, and the need for efficient handling of categorical variables and missing values, XGBoostClassifier stands out as a sound choice of classifier due to its robustness, speed, and theoretical underpinnings in gradient boosting techniques.

1. Handling Imbalanced Data: XGBoost is adept at handling imbalanced datasets, making it suitable for our situation in which severe accidents are rare compared to less severe ones. Its algorithm is designed to minimize loss while considering class imbalances, thus improving performance on minority classes.
2. Speed and Efficiency: XGBoost is known for its speed and efficiency, making it feasible to train on large datasets with numerous features. This is beneficial considering our dataset's size and the complexity of the classification task.
3. Handling Missing Values: XGBoost can handle missing values internally, reducing the preprocessing burden. Since our dataset contains missing values for most variables, this capability is advantageous and can lead to more robust models.
4. Feature Interaction Handling: While it's challenging to discern which variables might interact in our dataset, XGBoost's ensemble nature inherently captures interactions between features. This can lead to more nuanced and accurate predictions compared to simpler models.
5. Theoretical Basis for Gradient Boosting: Gradient boosting methods like XGBoost are theoretically sound and have been widely adopted in various domains due to their ability to iteratively improve model performance by focusing on the residuals of previous models. This iterative learning process can lead to strong predictive power even in complex, high-dimensional datasets like ours.

On the other hand, explainability and interpretability is very important when it comes to preventing severe traffic accidents. Interpreting XGBoost feature importance scores directly can be complex due to the ensemble nature of the model.

**Metric.** Misclassifying a non-severe accident as severe may divert resources from actual severe accidents, but misclassifying a severe accident as non-severe is worse—as it is potentially life-threatening. Thus, for each accident class, class-wise precision and recall are both important; however, for the most severe accident class, recall is more important, while for the least severe accident class (material damage only), precision is more important. For the middling severity class (minor accidents), let us say that precision and recall are of equal importance.

To elaborate: for the **most severe** accident class, class-wise precision and recall are both important, but **recall is more important**. Misclassifying a non-severe accident as severe may divert resources from actual severe accidents (precision is important), but misclassfying a severe accident as non-severe is worse as it is potentially life-threatening (recall is more important). It's vital to minimize false negatives (severe accidents misclassified as non-severe), even if it means some false positives (non-severe accidents misclassified as severe), as our aim is for all potentially dangerous situations to be attended to urgently. 

By the same token, for the **least severe accident** class (material damage only), **precision is more important**. In this class, a false positive (misclassifying a severe accident as non-severe) is worse, as it means a failure to recognize the actual danger posed. We therefore aim to minimize false positives, even at the expense of more false negatives, to ensure serious situations receive the attention they need.

Middling-severity class (minor accidents) can be misclassified as more severe (misallocation of resources), or less severe (potentially dangerous), so we'll put equal emphasis on recall and precision here.

Therefore, a class-wise $F_{\beta}$ score (weighted harmonic mean of precision and recall) seems an appropriate metric for evaluating the performance of a model on each accident class: $\beta > 1$ gives more weight to recall, $\beta < 1$ gives more weight to precision, and $\beta = 1$ gives equal weight to both. We'll choose $\beta = 2$ for the most severe accident class, $\beta = 1/2$ for the least severe accident class (material damage only), and $\beta = 1$ for the middling-severity class (minor accidents).

To evaluate the overall performance of a model (across all classes), we'll use a weighted average of the class-wise $F_{\beta}$ scores, which we'll denote by $F^{*}$:

\begin{equation}
 F^{*} = \frac{1}{w_0 + w_1 + w_2}\left(w_0F^{(0)}_{1/2} + w_1F^{(1)}_1 + w_2F^{(2)}_2\right), \tag{$*$}
\end{equation}

where $F^{(k)}_{\beta}$ denotes the $F_{\beta}$ score for severity class $k$, and each $w_k$ denotes a nonnegative weight. For the weights, recall that the dataset is highly imbalanced, with class size becoming drastically smaller with severity. A model trained on such data will likely have decent precision **and** recall in the least severe accident class, but very poor precision **and** recall in the most severe accident class. We would not want to say the model performed well overall just because it did well in the least severe class, if its recall in the most severe class is very low. Therefore, we should have $w_2/w_1 > 1$ and $w_1/w_0 > 1$. We choose 

\begin{equation}
 w_0 = 1, \quad w_1 = 4, \quad w_2 = 16.
\end{equation}

<a id='data'></a>
## Data
↑↑ [Contents](#contents) ↑ [Choice of classifier and metric](#choice_classifier) ↓ [Setup and loading data](#setup_and)

SOCIÉTÉ DE L'ASSURANCE AUTOMOBILE DU QUÉBEC (SAAQ). Rapports d'accident, [Jeu de données], dans Données Québec, 2017, mis à jour le 18 decembre 2023. https://www.donneesquebec.ca/recherche/dataset/rapports-d-accident.

_Données issues des rapports d’accident remplis par les policiers, incluant notamment le moment, la gravité de l’accident de même que le type des véhicules impliqués._

QUEBEC AUTOMOBILE INSURANCE SOCIETY (SAAQ). Accident reports, [Dataset], in Data Quebec, 2017, updated December 18, 2023. https://www.donneesquebec.ca/recherche/dataset/rapports-d-accident, (accessed March 13, 2024).

_Data from accident reports completed by police officers, including the time, severity of the accident as well as the type of vehicles involved._

<a id='setup_and'></a>
## Setup and loading data
↑↑ [Contents](#contents) ↑ [Data](#data) ↓ [Filtering and train/test split](#filtering)

We repeat the setup, loading, and filtering and train/test split process of our [exploratory data analysis](./01-data-exploration-saaq.ipynb).

In [1]:
from pathlib import Path
import os
import sys

# Determine the project root directory and add it to the Python path
notebook_path = Path(os.getcwd()).resolve()  # Path to the current working directory
project_root = notebook_path.parent          # Parent directory of notebooks, which is the project root

# Add the project root directory to the Python path
sys.path.append(str(project_root))

# The setup module is project_root/scripts/prjct_setup.py
from scripts.setup import *


PROJECT DIRECTORY STRUCTURE

├─ data/
├─ expository/
├─ literature/
├─ notebooks/
├─ scripts/
├─ models/
├─ presentation/
├─ streamlit/


PATHS TO FIRST-LEVEL SUBDIRECTORIES CONVENIENTLY STORED IN 'PATH' DICTIONARY

path['data'] = F:\projects\road-safety\data
path['expository'] = F:\projects\road-safety\expository
path['literature'] = F:\projects\road-safety\literature
path['notebooks'] = F:\projects\road-safety\notebooks
path['scripts'] = F:\projects\road-safety\scripts
path['models'] = F:\projects\road-safety\models
path['presentation'] = F:\projects\road-safety\presentation
path['streamlit'] = F:\projects\road-safety\streamlit


In [2]:
# Load csv files into one big dataframe
from processing import primary

data_dir: Path = path["data"]                 # Path to directory containing csv files
years: list = list(range(1999,2024))          # We want all data for years in this list, which does not have to be a subset of the years for which we actually hve data.
filename_format: str = "saaq_yyyy.csv"        # Or: "saaq_yyyy_fr.csv" (French version); "ncdb_yyyy.csv" (Canada-wide dataset).
Print: bool = False                           # If True, print value-count summary of dataframe.

saaq = primary(data_dir=data_dir,
               years=years,
               filename_format=filename_format,
               Print=Print,)


FOR EACH YEAR WE

  1. Read csv into dataframe.
  2. Replace strings 'x' by x if x is a number. May take a minute.
  3. Concatenate resulting dataframe with previous years' dataframe.

2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 
SOURCE DATAFRAME: self.df


<a id='filtering'></a>
## Filtering and train/test split
↑↑ [Contents](#contents) ↑ [Setup and loading data](#setup_and) ↓ [Creating an instance of the model class](#creating_an)

In [3]:
from typing import Type, Union
import pandas as pd

source: Type[primary] = saaq                            # Source data.
restrict_to: Union[dict, None] = {"REGION" :            # Remove all records *unless* column k lies in list v, for k : v in restrict_to dictionary.
                                 ["Montréal (06)",],}    
    
remove_if: Union[dict,None] = None                      # Remove all records if column k lies in list v, for k : v in remove_if dictionary.    
drop_row_if_missing_value_in: Union[list, None] = None  # We drop all rows for which there is a missing value (i.e. sentinel value, if applicable) in a column from this list.   
                                                        # Should include targets in this list (no point having missing target values). 
                                                        # Also, any column in "stratify_by" below, otherwise the stratified train-test-split will throw an error (TypeError: '<' not supported between instances of 'float' and 'str') if the values are strings.
targets: list = ["TNRY_SEV"]                            # We intend to make predictions for values in each of these columns.
                                                        # Can add 'MULT_VEH', 'VICTIMS', and 'TNRY_SEV' to targets (saaq data): columns will be inserted automatically.
                                                        # Can add 'MULT_VEH', 'VICTIMS' to targets (ncdb data).
non_features: list = ["YEAR"]                           # Can add "ID" etc. here but "ID" and any target will automatically be removed from features 
features: list = [c for c in saaq.df.columns            # These are the features we will want to use to make predictions about the targets.
                    if c not in non_features]            
                  
test_size: float = 0.15                                 # We'll partition what's left of the rows into train/test sets, with this as relative test set size.
                                                        # We'll also split the 'train' set later, either through k-fold cross-validation or, if not, train will be split 90/10 with the 10% share going to validation.
seed: int = 0                                           # Wherever there is randomness (e.g. during train/test split we'll shuffle), we'll use this seed. There are _some_ non-critical places where we introduce randomness without a seed.
stratify: bool = True                                   # If True, we'll perform a stratified train/test split. Also applies to k-fold cross-validation (if applicable).
stratify_by: Union[list, None] = targets                # If stratify is True, we'll stratify train/test split so that proportions are maintained for tuples of values from columns in this list.

We will work with an instance of the ```process``` class, which we call ```mtl_3sev``` (Montreal, three-level severity).

In [4]:
from processing import process

mtl_3sev = process(source=source,
                   restrict_to=restrict_to,
                   remove_if=remove_if,
                   drop_row_if_missing_value_in=drop_row_if_missing_value_in,
                   targets=targets,
                   features=features,
                   test_size=test_size,
                   seed=seed,
                   stratify=stratify,
                   stratify_by=stratify_by)

Removing ID from self.features

Removing all records unless:
  REGION in ['Montréal (06)']

Inserting 'TNRY_SEV' column.

Removing REGION from self.features (but not from self.df) as the number of distinct non-null values in self.df['REGION'] is 1.

Removing SEVERITY from self.features (but not from self.df): can't use SEVERITY to predict TNRY_SEV.

Removing NUM_VICTIMS from self.features (but not from self.df): can't use NUM_VICTIMS to predict TNRY_SEV.

Partitioning data into training/test sets: self.df_train/self.df_test.

self.ordinal_features = ['NUM_VEH', 'LIGHT', 'SPD_LIM']

self.ordinal_targets = ['TNRY_SEV']

self.categorical_features = ['WKDY_WKND', 'PED', 'MTRCYC', 'ACCDN_TYPE', 'BICYC', 'HOUR', 'LT_TRK', 'RD_CONFG', 'ASPECT', 'LONG_LOC', 'ZONE', 'RD_COND', 'RDWX', 'WEATHER', 'HVY_VEH', 'MONTH', 'PUB_PRIV_RD']

self.categorical_targets = []


<a id='creating_an'></a>
## Creating an instance of the model class
↑↑ [Contents](#contents) ↑ [Filtering and train/test split](#filtering) ↓ [A first model (XGBoost)](#first_model)

In our custom ```models``` module we have created a class called ```model```, attributes of which include ```data``` (an instance of the ```process``` class), ```classifier```, which can be any of five different kinds of classifier, ```folds```, which will be the $k$ in $k$-fold cross-validation, and so on. The attribute ```balance``` is set to ```None``` initially, but [later](#simple_oversampling) will cap the number of times a record from a minority class can be upsampled during training. The variables ```beta``` and ```weights``` store arrays of floats used to calculate our metric [($*$)](#choice_classifier): we are interested in the weighted average of class-wise $F_{\beta}$ scores with $\beta = $ ```beta[i]``` and weights $w_i = $ ```weights[i]```, for $i = 0,1,2$. The ```threshold_dict_help``` and ```threshold_dict_hinder``` attributes are set to ```None``` initially, but will enter the scene when we expirement with [adjusting classification thresholds](#adjusting_).

In [5]:
from models import model
from typing import Optional, Type, Union, Callable
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import numpy as np
from collections import OrderedDict

data: Type[process] = mtl_3sev    
folds: Union[int, None] = 5 
impute_strategy: Union[dict, None] = None # {'categorical': 'most_frequent', 'ordinal': 'most_frequent', 'constant' : -1}
classifier: Union[DecisionTreeClassifier,
                  GradientBoostingClassifier,
                  LogisticRegression, 
                  RandomForestClassifier, 
                  XGBClassifier,] = XGBClassifier(objective='multi:softmax', # 'binary:logistic', if two classes
                                                  eval_metric='mlogloss',    # 'logloss', if two classes
                                                  max_delta_step=1,                                                   
                                                  importance_type='weight', 
                                                  max_depth = 6, 
                                                  n_estimators = 100, 
                                                  nthread=-1,)
balance: Union[int, None] = None # See 'Simple oversampling' and 'Augmented oversampling' below
filename_stem: str = "mtl_3sev"
model_dir: Path = path["models"]   
beta: Union[np.ndarray,None] = np.array([0.5, 1, 2])
weights: Union[np.ndarray,None] = np.array([1,4,16])
threshold_dict_help: Union[None, OrderedDict[Union[int, str], float]] = None    # See 'Adjusting classification thresholds' below
threshold_dict_hinder: Union[None, OrderedDict[Union[int, str], float]] = None  # See 'Adjusting classification thresholds' below

<a id='first_model'></a>
## A first model (XGBoost)
↑↑ [Contents](#contents) ↑ [Creating an instance of the model class](#creating_an) ↓ [Adjusting classification thresholds](#adjusting_)

We can pass these attributes to the ```model``` class to create an instance, which we name ```mtl_3sev_xgb```. The printed output will explain what is happening.

In [6]:
mtl_3sev_xgb = model(data=data,    
                     folds=folds, 
                     impute_strategy=impute_strategy, 
                     classifier=classifier,
                     balance=balance,
                     filename_stem=filename_stem,
                     model_dir=model_dir,
                     beta=beta,
                     weights=weights,
                     threshold_dict_help=threshold_dict_help,
                     threshold_dict_hinder=threshold_dict_hinder,)


SEPARATING FEATURES FROM TARGETS

self.X_train/self.X_test, self.y_train/self.y_test

MAPPING ORDINAL FEATURE/TARGET CODES

NUM_VEH: {1: 0, 2: 1, 9: 2}
LIGHT: {1: 3, 2: 2, 3: 1, 4: 0}
SPD_LIM: {'<50': 0, 50: 5, 60: 6, 70: 7, 80: 8, 90: 9, 100: 10}
TNRY_SEV: {0: 0, 1: 1, 2: 2}

MAPPING CATEGORICAL FEATURE/TARGET CODES

WKDY_WKND: {'WKDY': 0, 'WKND': 1}
PED: {'N': 0, 'Y': 1}
MTRCYC: {'N': 0, 'Y': 1}
ACCDN_TYPE: {'vehicle': 0, 'pedestrian': 1, 'cyclist': 2, 'animal': 3, 'fixed object': 4, 'no collision': 5, 'other': 6}
BICYC: {'N': 0, 'Y': 1}
HOUR: {'00:00:00-03:59:00': 0, '04:00:00-07:59:00': 1, '08:00:00-11:59:00': 2, '12:00:00-15:59:00': 3, '16:00:00-19:59:00': 4, '20:00:00-23:59:00': 5}
LT_TRK: {'N': 0, 'Y': 1}
RD_CONFG: {1: 0, 23: 1, 45: 2, 9: 3}
ASPECT: {'Straight': 0, 'Curve': 1}
LONG_LOC: {12: 0, 33: 1, 34: 2, 40: 3, 69: 4, 99: 5}
ZONE: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 9: 6}
RD_COND: {11: 0, 12: 1, 13: 2, 14: 3, 15: 4, 16: 5, 17: 6, 18: 7, 19: 8, 20: 9, 99: 10}
RDWX: {'N': 0,

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal',
                                                  OrdinalEncoder(categories=[[0,
                                                                              1,
                                                                              2],
                                                                             [0,
                                                                              1,
                                                                              2,
                                                                              3],
                                                                             [0,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9,
                                                                              10]],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=nan),
                                                  ['NUM_VEH', 'LIGHT',
                                                   'SPD_LIM']),
                                                 ('onehot',
                                                  OneHotEncoder(categories=[[0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type='weight',
                               interaction_constraints=None, learning_rate=None,
                 


       MAKING PREDICTIONS ON VALIDATION FOLD

CV 1:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 2:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 3:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 4:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

Target, prediction, and probabilities for each validation fold stored in self.preds dictionary.

Feature importances corresponding to each validation fold stored in self.feature_importances dictionary.

SAVING PIPELINE

F:\projects\road-safety\models\mtl_3sev_xgb.joblib

Metrics (including confusion matrices) contained in dictionary self.eval
Metrics (excluding confusion matrices) also contained in dataframe self.eval_df

OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.854689,0.450485,0.662601,0.450485,0.476317,0.456741,0.480859,0.843630
1,0.854279,0.451579,0.658248,0.451579,0.479657,0.458345,0.478932,0.844434
2,0.855953,0.453239,0.634036,0.453239,0.480596,0.459941,0.486516,0.845408
3,0.855401,0.448354,0.605800,0.448354,0.473124,0.454243,0.483662,0.843803
4,0.855487,0.450496,0.620162,0.450496,0.476467,0.456761,0.484229,0.840917
mean,0.855162,0.450831,0.636169,0.450831,0.477232,0.457206,0.482840,0.843638



CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


Feature importances corresponding to validation set(s) contained in self.feature_importances and self.feature_importances_df.


FEATURE IMPORTANCE



interactive(children=(Dropdown(description='Sort by:', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_clas…

Although $F^*$ is the one metric we will use to evaluate our model, our ```model``` class also outputs global metrics (as can be seen above), including accuracy ```ACC```, balanced accuracy ```BACC```, precision, recall, $F_1$, $F_2$, Matthews Correlation Coefficient ```MCC```, and One-vs-Rest (OvR) Receiver Operating Characteristic - Area Under the Curve ```ROC-AUC```. The global precision, recall, and $F$-scores use macro-averaging and so give equal weight to each severity class, which is not what we want.

As can be seen from the confusion matrices, the model correctly classifies, on average, around 4 out of the 484 most severe accidents represented in the validation folds. Our overall evaluation metric $F^*$ (a weighted average of the class-wise $F_{\beta}$ scores—see [($*$)](#choice_classifier)) is shown in the bottom right corner of the matrix. It is very low, due in large part to the poor class-wise recall in the minor and severe classes, especially the latter. 

We will experiment with methods to enhance our model's performance with respect to our $F^*$ metric. We first explore the impact of [adjusting classification thresholds](#adjusting_). Currently, our model predicts the severity level corresponding to the highest probability assigned by the classifier. However, we can modify the model so that an accident is classified as 'serious/fatal' if the probability for this category reaches at least, for instance, 40%, potentially increasing sensitivity to more critical cases.

Next, we investigate the effects of [upsampling the two most severe accident classes](#simple_oversamlping). We perform both [simple](#simple_oversampling) and [augmented](#augmented_oversampling) oversampling. It's important to recognize that while upsampling can help correct class imbalances, it may also alter the joint distribution of our data, potentially introducing biases in the model's predictions. We must carefully evaluate whether these adjustments lead to genuine performance improvements or if they skew the model's understanding of the underlying data distribution.

Finally, we try [combining](#combining) upsampling with adjustment of classification thresholds.

<a id='adjusting_'></a>
## Adjusting classification thresholds
↑↑ [Contents](#contents) ↑ [A first model (XGBoost)](#first_model) ↓ [Simple oversampling](#simple_oversampling)

The ```prediction``` function from our ```evaluation``` module is designed to classify accidents based on probabilities assigned by a classifier. It can optionally take two ```OrderedDict``` arguments: ```threshold_dict_help``` and ```threshold_dict_hinder```, which temporarily adjust the probabilities before determining the final prediction. It's important to note that these adjustments to probabilities are used only for determining the final prediction and do not permanently alter the original probabilities data.

To explain by example, let's assume we pass ```threshold_dict_help = OrderedDict([(2, 0.25), (1, 0.3)])``` to the function. This dictionary sets up the following rules:
- If the probability of class 2 exceeds 25%, then class 2 is immediately predicted.
- If not, but the probability of class 1 exceeds 30%, then class 1 is predicted.
- If neither condition is met, the class with the highest probability is chosen, which could still be class 2 or 1 depending on their probabilities relative to the thresholds and other class probabilities.

To give highest priority to class 1, so that it is immediately predicted if its probability exceeds 30%, we simply place it first in the ordered dictionary: ```threshold_dict_help = OrderedDict([(1, 0.3),(2, 0.25)])```.

Similarly, ```threshold_dict_hinder = OrderedDict([(0, 0.6)])``` implies that class 0 will only be considered if its probability exceeds 60% and it is the highest probability among the classes. Internally, the function works internally as follows. 

Adjustments Based on ```threshold_dict_help```:
  - Continuing with our example, each probability for class 2 that exceeds 25% is temporarily set to 100%.
  - If a class 2 probability does not exceed 25% but the probability for class 1 exceeds 30%, class 1's probability is temporarily set to 100%.

Adjustments Based on ```threshold_dict_hinder```:
  - Any probability for class 0 that (in our example) is at most 60% is temporarily set to 0%.

The function then identifies the class with the highest 'augmented' probability. For example:
  - Probabilities of 26%, 40%, 34% for classes 2, 1, and 0 respectively become 100%, 40%, 0%. The prediction will be class 2.
  - Probabilities of 24%, 40%, 36% become 24%, 100%, 0%. The prediction will be class 1.
  - Probabilities of 26%, 13%, 61% become 100%, 13%, 61%. The prediction will be class 2.

Again, these 'augmented' probabilities are created only as an intermediate step to facilitate the threshold adjustments. Ultimately, we maintain the original probabilities given by the classifier.

This function is versatile as both ```threshold_dict_help``` and ```threshold_dict_hinder``` can be used simultaneously, separately, or not at all, reverting to a default mode that simply picks the class with the maximum probability if no dictionaries are provided.


We wrap the ```prediction``` function inside the ```augmented_predictions``` function of our ```models``` module, which creates a copy of ```self.preds```, but with augmented probabilities and predictions based on adjusted thresholds. It also displays confusion matrices and metrics corresponding to these new predictions.

In [7]:
from collections import OrderedDict
from typing import Dict, List
from models import adjusted_predictions

instance = mtl_3sev_xgb

threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([(2,0.25), (1,0.3)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = OrderedDict([(0,0.6)]) 

adjusted_predictions(instance=instance,
                     threshold_dict_help=threshold_dict_help,
                     threshold_dict_hinder=threshold_dict_hinder,);


THRESHOLD ADJUSTMENTS: 'HELP' ORDEREDDICT({2: 0.25, 1: 0.3})', 'HINDER' ORDEREDDICT({0: 0.6})


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.822291,0.496130,0.555995,0.496130,0.505214,0.498592,0.444658,0.843630
1,0.823893,0.496104,0.546358,0.496104,0.506304,0.499122,0.445599,0.844434
2,0.821632,0.501494,0.546367,0.501494,0.510975,0.504217,0.445307,0.845408
3,0.823661,0.497704,0.548477,0.497704,0.507244,0.500433,0.447746,0.843803
4,0.823498,0.497268,0.550739,0.497268,0.507148,0.500099,0.446545,0.840917
mean,0.822995,0.497740,0.549587,0.497740,0.507377,0.500493,0.445971,0.843638


We see only a very slight improvement of the $F^*$ score. We can perform a grid search across various combinations of thresholds that 'help' the 'serious/fatal' and 'minor' accident classes (improving recall), while 'hindering' the 'material damage only' accident class (improving precision). We'll find the best score across helpful thresholds ranging over [0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45] for classes 2 and 1, and a hindering threshold ranging over [0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 ] for class 0. The resulting grid contains 729 points. This is achieved via the ```threshold_grid_search``` function of our custom ```models``` module. 

In [8]:
from typing import Tuple
from models import threshold_grid_search

instance: Type[model] = mtl_3sev_xgb

priority: List[Tuple[int, str]] = [(2,'help'), 
                                   (1,'help'), 
                                   (0,'hinder')]

threshold: np.ndarray = np.array([np.arange(0.05, 0.5, 0.05), 
                                  np.arange(0.05, 0.5, 0.05), 
                                  np.arange(0.5, 0.95, 0.05),])

timeout: Union[int, None] = None # Integer n means incomplete grid search will stop shortly after n seconds.

best_score, best_thresholds = threshold_grid_search(instance=instance,
                                                    priority=priority,
                                                    threshold=threshold,
                                                    timeout=timeout,)

Best mean F* score so far (F* averaged over validation folds): 0.29856073987215426
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.05}), hinder OrderedDict({0: 0.5})
Time taken: 0.49 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.313243617257942
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.1}), hinder OrderedDict({0: 0.5})
Time taken: 4.81 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.3222270810484951
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.15}), hinder OrderedDict({0: 0.5})
Time taken: 9.04 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.325875374102628
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.2}), hinder OrderedDict({0: 0.5})
Time taken: 13.32 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.32589830370474093
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.25}),

We could do a finer grid search around the best threshold values found above, but we won't obtain a significantly better $F^*$ score. We can add a new ```augmented_preds``` attribute to the instance of our model class we're working with, containing the predictions that give the best score found in our grid search.

In [9]:
instance = mtl_3sev_xgb

threshold_dict_help, threshold_dict_hinder = best_thresholds

instance.adjusted_preds = adjusted_predictions(instance=instance,
                                               threshold_dict_help=threshold_dict_help,               
                                               threshold_dict_hinder=threshold_dict_hinder,)


THRESHOLD ADJUSTMENTS: 'HELP' ORDEREDDICT({2: 0.05, 1: 0.25})', 'HINDER' ORDEREDDICT({0: 0.75})


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.763582,0.588094,0.472423,0.588094,0.490646,0.523814,0.364011,0.843630
1,0.765914,0.593389,0.475539,0.593389,0.494964,0.529166,0.367811,0.844434
2,0.762407,0.588268,0.470682,0.588268,0.488444,0.521742,0.360776,0.845408
3,0.764615,0.584896,0.473476,0.584896,0.492355,0.525056,0.366805,0.843803
4,0.763240,0.590906,0.473723,0.590906,0.493484,0.527776,0.363789,0.840917
mean,0.763952,0.589111,0.473169,0.589111,0.491979,0.525511,0.364638,0.843638


Let's place the original confusion matrix and the threshold-adjusted confusion matrix side by side for easy comparison.

In [10]:
from models import confusion_matrix_widget, confusion_matrix_with_weighted_fbeta

instance = mtl_3sev_xgb

threshold_dict_help, threshold_dict_hinder = best_thresholds

original_eval = instance.evaluate(prediction_dictionary=instance.preds)
adjusted_eval = instance.evaluate(prediction_dictionary=instance.adjusted_preds)

confusion_matrix_widget(original_eval, 
                        confusion_matrix_with_weighted_fbeta,
                        beta = instance.beta, 
                        weights = instance.weights,)

print_help_dict = OrderedDict((k, int(v * 100) / 100.0) for k, v in threshold_dict_help.items())
print_hinder_dict = OrderedDict((k, int(v * 100) / 100.0) for k, v in threshold_dict_hinder.items())
print_header(f"Below with threshold adjustments: \'help\' {print_help_dict}', \'hinder\' {print_hinder_dict}")

confusion_matrix_widget(adjusted_eval, 
                        confusion_matrix_with_weighted_fbeta,
                        beta = instance.beta, 
                        weights = instance.weights,)


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


BELOW WITH THRESHOLD ADJUSTMENTS: 'HELP' ORDEREDDICT({2: 0.05, 1: 0.25})', 'HINDER' ORDEREDDICT({0: 0.75})


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()

Despite implementing rather extreme threshold values, our overall $F^*$ metric remains disappointingly low—albeit double the non-threshold-adjusted score. Specifically, the number of serious/fatal accidents correctly identified has increased from 4 to over 200 out of 484, marking a noteworthy improvement. However, this is still far from adequate. There's a modest enhancement in recall for the minor accident class, and a small improvement in precision for the material damage class.

Furthermore, these gains come at a significant cost: the recall for class 0 has substantially worsened, as have the precision metrics for classes 1 and 2. 

This suggests that the classifier is overwhelmingly assigning very high probability to the majority class for most accidents, reflecting a bias towards more frequent outcomes. 

Additionally, the Matthews correlation coefficient (MCC) for the augmented predictions is now significantly closer to zero, indicating that the (augmented) classifier's performance is approaching that of random guessing. 

<a id='simple_oversampling'></a>
## Simple oversampling
↑↑ [Contents](#contents) ↑ [Adjusting classification thresholds](#adjusting_) ↓ [Augmented oversampling](#augmented_oversampling)

As can be seen from the output of the code block below, our training set exhibits significant class imbalance: there are 92 times more class 0 accidents than class 2 accidents, and the ratio of class 0 to class 1 accidents is 22. To address this, we aim to balance the classes by oversampling each class 2 record up to 92 times, and each class 1 record up to 22 times. We will experiment with varying degrees of oversampling for each class to determine the most effective approach.

In [11]:
instance = mtl_3sev_xgb
print(instance.y_train["TNRY_SEV"].value_counts())
ratio_class_0_to_1: int = instance.y_train["TNRY_SEV"].value_counts()[0]//instance.y_train["TNRY_SEV"].value_counts()[1]
ratio_class_1_to_2: int = instance.y_train["TNRY_SEV"].value_counts()[1]//instance.y_train["TNRY_SEV"].value_counts()[2]
ratio_class_0_to_2: int = instance.y_train["TNRY_SEV"].value_counts()[0]//instance.y_train["TNRY_SEV"].value_counts()[2]

print(f"\nClass 0 is more than {ratio_class_0_to_1} times the size of Class 1.")
print(f"Class 1 is more than {ratio_class_1_to_2} times the size of Class 2.")
print(f"Class 0 is more than {ratio_class_0_to_2} times the size of Class 2.")

TNRY_SEV
0    224129
1     54331
2      2419
Name: count, dtype: int64

Class 0 is more than 4 times the size of Class 1.
Class 1 is more than 22 times the size of Class 2.
Class 0 is more than 92 times the size of Class 2.


We're conducting 5-fold cross-validation: the code snippet below from the ```fit``` method of our models class illustrates how ```X_train``` and ```y_train``` are selected for each fold, before the model is fit to these datasets.

```
X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
```

```
self.pipeline.fit(X_train, np.ravel(y_train))
```

Prior to fitting the model, our ```balance``` function modifies ```X_train``` and ```y_train```, resulting in ```X_balance``` and ```y_balance```. This adjustment involves upsampling the minority classes to achieve a more balanced dataset, with the upsampling factor determined by the ```balance``` attribute of our ```models class```. This ensures that each class is adequately represented during the training, potentially improving model robustness and performance.

The ```balance``` function of our ```models``` module is designed to address class imbalance in training datasets by implementing a simple oversampling technique. In cases where certain classes are underrepresented, the function replicates the existing records from these minority classes up to a specified number of times, defined by the ```max_repeats``` parameter. This method aims to equalize the class distribution, ensuring that each class has a presence in the dataset that is more proportional to the majority class. The replication process stops either when the minority classes reach the size of the majority class or when the maximum specified repetitions are achieved. Importantly, this oversampling method is applied solely to the training set to prevent any data leakage and ensure that the validation and test sets remain untouched and representative of true, real-world distributions.

We illustrate use of the ```balance``` function in the code cell below.

In [12]:
from models import explain_balance
from typing import Type

instance: Type[model]
max_repeats: int = 25

explain_balance(instance=instance,
                max_repeats=max_repeats,)


EXAMPLE OF SIMPLE OVERSAMPLING

- This process adjusts the training dataset to balance class distribution.
- It increases the representation of minority classes.
- Records are repeated until their number matches either the majority class, or has increased 25-fold, whichever results in fewer repetitions.

FOLD 0

Time taken to generate 'balanced' training data: 3.00 seconds.

ORIGINAL TRAINING SET CLASS COUNTS:


TNRY_SEV
0           179304
1            43464
2             1935
Name: count, dtype: int64


UPSAMPLED TRAINING SET CLASS SIZES:


TNRY_SEV
0           179304
1           179304
2            48375
Name: count, dtype: int64


Class 0: 179304 records repeated 1 times, 
Class 1: 38016 records repeated 4 times, 5448 records repeated 5 times, 
Class 2: 1935 records repeated 25 times, 

Note that the validation set has NOT been altered.

VALIDATION SET CLASS COUNTS:


TNRY_SEV
0           44825
1           10867
2             484
Name: count, dtype: int64


Press enter to continue or enter c to cancel: c


Note that the way we oversample ensures that, in a given minority class, all but at most a few records are sampled the same number of times, and those few (if any) are sampled only one more time than the rest. This uniformity helps prevent bias, maintains fair representation across the dataset, and reduces the risk of overfitting by ensuring that no single record disproportionately influences the model, thereby enhancing the model's ability to generalize to new data.

In our approach, we will conduct a customized 'partial grid search' to fine-tune our model by testing various combinations of settings for the ```balance``` attribute and the ```max_depth``` and ```n_estimators``` parameters of the ```XGBClassifier```.

We note that while ```GridSearchCV``` can be used for an extensive search of classifier parameters by setting the ```param_grid``` and ```grid_search_scoring``` attributes in a ```model``` class instance, it does not automatically address class imbalances through upsampling.

The code cell below demonstrates how we create instances of our model class using different settings for the ```balance``` attribute, along with the ```max_depth``` and ```n_estimators``` parameters of the ```XGBClassifier```, which is incorporated through the ```classifier``` attribute of our ```model``` class.

We use the ```save_gs_items``` function to save the global metrics and confusion matrices, which are stored in a dictionary, to a ```.txt``` file. Conversely, the ```load_gs_items``` function retrieves this data. This system checks if a particular combination of (```balance```, ```max_depth```, ```n_estimators```) already exists in the ```.txt``` file. If it does, the model with that specific combination is not rerun, allowing us to efficiently skip to the next combination.

In [13]:
from models import custom_xgb_grid_search

instance: Type[model] = mtl_3sev_xgb
balance_range: Union[List, np.array] = [1] + [10*i for i in range(1,10)]
max_depth_range: Union[List, np.array] =  [3,6]
n_estimators_range: Union[List, np.array] = [100,150]
results_filename: str = "mtl_3sev_gs"                           
timeout: Union[int, None] = None

custom_xgb_grid_search(instance=instance, 
                       balance_range=balance_range,
                       max_depth_range=max_depth_range,
                       n_estimators_range=n_estimators_range,
                       results_filename=results_filename,                           
                       timeout=timeout,)

Time taken: 50.0 minutes, 21.29 seconds.
Cumulative data for 40 of 40 grid points will now be stored in a .txt file.
File saved successfully to: F:\projects\road-safety\models\mtl_3sev_gs.txt


We can use the ```grid_search_results``` function from our ```models``` module to display key metrics corresponding to each grid point, and sort according to a chosen metric.

In [13]:
from models import grid_search_results
from typing import Type, Union

instance: Type[model] = mtl_3sev_xgb
results_filename: str = "mtl_3sev_gs"
sort_by: Union[None, str] = "F*"

gs_results = grid_search_results(instance=instance,
                                 results_filename=results_filename,
                                 sort_by=sort_by)
display(gs_results)

,F*,precision 0,precision 1,precision 2,recall 0,recall 1,recall 2
"(70, 6, 150)",0.288247,0.919075,0.387700,0.074739,0.758310,0.585404,0.430642
"(70, 6, 100)",0.288247,0.919075,0.387700,0.074739,0.758310,0.585404,0.430642
"(70, 3, 150)",0.288247,0.919075,0.387700,0.074739,0.758310,0.585404,0.430642
"(70, 3, 100)",0.288247,0.919075,0.387700,0.074739,0.758310,0.585404,0.430642
"(60, 6, 150)",0.288102,0.919923,0.394745,0.079086,0.759358,0.613841,0.372671
"(60, 6, 100)",0.288102,0.919923,0.394745,0.079086,0.759358,0.613841,0.372671
"(60, 3, 150)",0.288102,0.919923,0.394745,0.079086,0.759358,0.613841,0.372671
"(60, 3, 100)",0.288102,0.919923,0.394745,0.079086,0.759358,0.613841,0.372671
"(50, 3, 100)",0.288083,0.919297,0.400900,0.089971,0.760831,0.639426,0.325052
"(50, 3, 150)",0.288083,0.919297,0.400900,0.089971,0.760831,0.639426,0.325052


We display confusion matrices and other metrics for any grid point, using the ```grid_point_summary``` function of our ```models``` module. We select the grid point yielding the highest $F^*$ score by passing ```'F*'``` to the ```sort_by``` parameter of the ```grid_search_results``` function, which we have stored in the variable ```gs_results```. If there are multiple grid points that give rise to the maximal $F^*$ score, we select the grid point with smallest ```max_depth``` and ```n_estimators``` coordinates. We pass ```gs_results.index[idx]```, for ```idx``` corresponding to the optimal grid point, to the ```gs_params``` parameter of the ```grid_point_summary``` function.

In [14]:
from models import grid_point_summary
from typing import Type

instance: Type[model] = mtl_3sev_xgb
results_filename = "mtl_3sev_gs"
gs_params = gs_results.index[3]

grid_point_summary(instance=instance,
                   results_filename=results_filename,
                   gs_params=gs_params)


(BALANCE, MAX_DEPTH, N_ESTIMATORS) = (70, 3, 100)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.721109,0.583951,0.458693,0.583951,0.471171,0.506494,0.353693,0.829886
1,0.718866,0.580535,0.458079,0.580535,0.470542,0.506160,0.354046,0.830113
2,0.719489,0.577913,0.456885,0.577913,0.468626,0.503083,0.351791,0.830723
3,0.719613,0.588449,0.459145,0.588449,0.471629,0.507916,0.354399,0.830503
4,0.722047,0.591452,0.460505,0.591452,0.474943,0.512533,0.353874,0.828806
mean,0.720225,0.584460,0.458661,0.584460,0.471382,0.507237,0.353560,0.830006



CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()

To create and save an actual model with these parameters, we need to pass them to our ```model``` class to create a new instance of it, which we call ```mtl_3sev_simple_oversample```.

In [15]:
import re 
gs_params = gs_results.index[3]
coords = re.findall(r'\d+', gs_params)
balance, max_depth, n_estimators  = [int(coord) for coord in coords]

print_header(f"Simple oversample up to {balance} times, max_depth = {max_depth}, n_estimaters = {n_estimators}")

data: Type[process] = mtl_3sev    
folds: Union[int, None] = 5 
impute_strategy: Union[dict, None] = None # {'categorical': 'most_frequent', 'ordinal': 'most_frequent', 'constant' : -1}
classifier: Union[DecisionTreeClassifier,
                  GradientBoostingClassifier,
                  LogisticRegression, 
                  RandomForestClassifier, 
                  XGBClassifier,] = XGBClassifier(objective='multi:softmax', # 'binary:logistic', if two classes
                                                  eval_metric='mlogloss',    # 'logloss', if two classes
                                                  max_delta_step=1,                                                   
                                                  importance_type='weight', 
                                                  max_depth = max_depth, 
                                                  n_estimators = n_estimators, 
                                                  nthread=-1,)
balance: Union[int, None] = balance
filename_stem: str = "mtl_3sev_simple_oversample"
model_dir: Path = mtl_3sev_xgb.model_dir  
beta: Union[np.ndarray,None] = mtl_3sev_xgb.beta
weights: Union[np.ndarray,None] = mtl_3sev_xgb.weights

mtl_3sev_simple_oversample_xgb = model(data=data,    
                                       folds=folds, 
                                       impute_strategy=impute_strategy, 
                                       classifier=classifier,
                                       balance=balance,
                                       filename_stem=filename_stem,
                                       model_dir=model_dir,
                                       beta=beta,
                                       weights=weights,)


SIMPLE OVERSAMPLE UP TO 70 TIMES, MAX_DEPTH = 3, N_ESTIMATERS = 100


SEPARATING FEATURES FROM TARGETS

self.X_train/self.X_test, self.y_train/self.y_test

MAPPING ORDINAL FEATURE/TARGET CODES

NUM_VEH: {1: 0, 2: 1, 9: 2}
LIGHT: {1: 3, 2: 2, 3: 1, 4: 0}
SPD_LIM: {'<50': 0, 50: 5, 60: 6, 70: 7, 80: 8, 90: 9, 100: 10}
TNRY_SEV: {0: 0, 1: 1, 2: 2}

MAPPING CATEGORICAL FEATURE/TARGET CODES

WKDY_WKND: {'WKDY': 0, 'WKND': 1}
PED: {'N': 0, 'Y': 1}
MTRCYC: {'N': 0, 'Y': 1}
ACCDN_TYPE: {'vehicle': 0, 'pedestrian': 1, 'cyclist': 2, 'animal': 3, 'fixed object': 4, 'no collision': 5, 'other': 6}
BICYC: {'N': 0, 'Y': 1}
HOUR: {'00:00:00-03:59:00': 0, '04:00:00-07:59:00': 1, '08:00:00-11:59:00': 2, '12:00:00-15:59:00': 3, '16:00:00-19:59:00': 4, '20:00:00-23:59:00': 5}
LT_TRK: {'N': 0, 'Y': 1}
RD_CONFG: {1: 0, 23: 1, 45: 2, 9: 3}
ASPECT: {'Straight': 0, 'Curve': 1}
LONG_LOC: {12: 0, 33: 1, 34: 2, 40: 3, 69: 4, 99: 5}
ZONE: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 9: 6}
RD_COND: {11: 0, 12: 1, 13: 2, 1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal',
                                                  OrdinalEncoder(categories=[[0,
                                                                              1,
                                                                              2],
                                                                             [0,
                                                                              1,
                                                                              2,
                                                                              3],
                                                                             [0,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9,
                                                                              10]],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=nan),
                                                  ['NUM_VEH', 'LIGHT',
                                                   'SPD_LIM']),
                                                 ('onehot',
                                                  OneHotEncoder(categories=[[0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type='weight',
                               interaction_constraints=None, learning_rate=None,
                 


       MAKING PREDICTIONS ON VALIDATION FOLD

CV 1:  FOLDING, BALANCING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 2:  FOLDING, BALANCING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 3:  FOLDING, BALANCING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 4:  FOLDING, BALANCING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

Target, prediction, and probabilities for each validation fold stored in self.preds dictionary.

Feature importances corresponding to each validation fold stored in self.feature_importances dictionary.

SAVING PIPELINE

F:\projects\road-safety\models\mtl_3sev_simple_oversample_xgb.joblib

Metrics (including confusion matrices) contained in dictionary self.eval
Metrics (excluding confusion matrices) also contained in dataframe self.eval_df

OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.707099,0.600131,0.450394,0.600131,0.458174,0.493193,0.328270,0.823893
1,0.708897,0.614822,0.453823,0.614822,0.463011,0.500870,0.335171,0.825069
2,0.707224,0.605580,0.449947,0.605580,0.457902,0.493653,0.326355,0.825972
3,0.709502,0.614826,0.453713,0.614826,0.462967,0.500541,0.333510,0.825211
4,0.708981,0.610635,0.453480,0.610635,0.463051,0.500575,0.333100,0.822756
mean,0.708341,0.609199,0.452271,0.609199,0.461021,0.497766,0.331281,0.824580



CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


Feature importances corresponding to validation set(s) contained in self.feature_importances and self.feature_importances_df.


FEATURE IMPORTANCE



interactive(children=(Dropdown(description='Sort by:', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_clas…

We see that simple oversampling has a very similar effect as threshold adjustment on our model's performance on the validation set. In fact, the results here are slightly worse. 

<a id='augmented_oversampling'></a>
## Augmented oversampling
↑↑ [Contents](#contents) ↑ [Simple oversampling](#simple_oversampling) ↓ [Combining threshold adjustments and oversampling](#combining)

We started with a dataset of more than 1.7 million police records across all of Québec, but narrowed our focus to just over 300,000 records within the Montréal region. Instead of merely replicating Montréal records to oversample minority classes, we now consider upsampling from the extensive dataset outside of Montréal. Given the reasonable assumption that the distribution of accident severities and the nature of accidents are similar across most regions in Québec—with potential variations due to factors like population density and urban versus rural settings—augmented oversampling could be advantageous. Unlike simple oversampling, augmented oversampling introduces diversity while aiming to maintain the class characteristics, potentially leading to better model generalizability and reduced overfitting.

We'll begin by making a deep copy of our ```mtl_3sev_xgb``` model instance, which we call ```mtl_3sev_xgb```.

In [16]:
import copy

mtl_3sev_xgb_copy = copy.deepcopy(mtl_3sev_xgb)
mtl_3sev_xgb_copy.filename_stem = 'mtl_3sev_augmented_oversample'

Next, we employ the ```create_supplementary_data``` function from our ```models``` module, which initiates by excluding any records previously selected for the initial dataset. This approach ensures that the supplementary dataset formed contains no overlapping records with the original dataset, maintaining distinct data pools. As a result, two new attributes, ```data.supplement.X_train``` and ```data.supplement.y_train```, are generated and designated as supplementary data for oversampling purposes. These attributes are specifically tailored to enrich our training data without reintroducing previously utilized records.

In [17]:
from models import create_supplementary_data

instance: Type[model] = mtl_3sev_xgb_copy
target = instance.y_train.columns[0]
majority_class = int(instance.y_train[target].value_counts().argmax())
remove_class: Union[list, None] = [majority_class]

create_supplementary_data(instance=instance,
                          remove_class=remove_class,)


CREATING SUPPLEMENTARY DATA FOR OVERSAMPLING

- We extract unused records from source data, following the same initial steps as with current model data.
- Note that what we remove here is precisely what we restricted to earlier.
- Thus, there is no overlap whatsoever between supplementary data and current model data.
- Dummy 'test' sets are only created to avoid errors. They contain only 3 records and can be ignored.

Removing all records if:
  REGION in ['Montréal (06)']

Inserting 'TNRY_SEV' column.

Partitioning data into training/test sets: self.df_train/self.df_test.

self.ordinal_features = ['NUM_VEH', 'LIGHT', 'SPD_LIM']

self.ordinal_targets = ['TNRY_SEV']

self.categorical_features = ['WKDY_WKND', 'PED', 'MTRCYC', 'ACCDN_TYPE', 'BICYC', 'HOUR', 'LT_TRK', 'RD_CONFG', 'ASPECT', 'LONG_LOC', 'ZONE', 'RD_COND', 'RDWX', 'WEATHER', 'HVY_VEH', 'MONTH', 'PUB_PRIV_RD']

self.categorical_targets = []

SEPARATING FEATURES FROM TARGETS

self.X_train/self.X_test, self.y_train/self.y_test



Let us investigate the class distributions of the original data, the supplementary data, and the combined data. We see that, in the combined training data, Class 0 and Class 1 are represented equally, and while Class 2 remains much smaller, its relative size has increased almost by a factor of 10.

In [18]:
instance: Type[model] = mtl_3sev_xgb_copy

print("Original data class sizes:".upper())
print(instance.y_train["TNRY_SEV"].value_counts())
print("\nSupplementary data class sizes:".upper())
print(instance.data.supplement.y_train["TNRY_SEV"].value_counts())

combined_class_0 = instance.y_train["TNRY_SEV"].value_counts()[0]
combined_class_1 = instance.y_train["TNRY_SEV"].value_counts()[1] + instance.data.supplement.y_train["TNRY_SEV"].value_counts()[1]
combined_class_2 = instance.y_train["TNRY_SEV"].value_counts()[2] + instance.data.supplement.y_train["TNRY_SEV"].value_counts()[2]

ratio_class_1_to_0: int = combined_class_1/combined_class_0
ratio_class_1_to_2: int = combined_class_1//combined_class_2
ratio_class_0_to_2: int = combined_class_0//combined_class_2

print(f"\nCombined Class 1 has size {combined_class_1}, the same size as Combined Class 0.")
print(f"Combined Class 0 and Combined Class 1 are just over {ratio_class_0_to_2} times larger than Combined Class 2, which is of size {combined_class_2}.")

ORIGINAL DATA CLASS SIZES:
TNRY_SEV
0    224129
1     54331
2      2419
Name: count, dtype: int64

SUPPLEMENTARY DATA CLASS SIZES:
TNRY_SEV
1    169798
2     16395
Name: count, dtype: int64

Combined Class 1 has size 224129, the same size as Combined Class 0.
Combined Class 0 and Combined Class 1 are just over 11 times larger than Combined Class 2, which is of size 18814.


We employ the same ```balance``` function from our ```models``` module as used previously, now tailored to integrate our instance's new 'supplementary' data attribute. This function combines the original and supplementary training data, from which it uniformly samples each record within a given class a specified number of times to achieve balance. If additional samples are required to meet the desired class size after this initial uniform sampling, the function randomly selects the remaining records from the combined dataset. This method ensures that each class reaches the target size while maintaining a balanced representation from both data sources.

In [19]:
from models import explain_balance
from typing import Type

instance: Type[model] = mtl_3sev_xgb_copy
max_repeats: int = 25
augmented: Union[None, bool] = True

explain_balance(instance=instance,
                max_repeats=max_repeats,
                augmented=augmented,)


EXAMPLE OF AUGMENTED OVERSAMPLING

- This process adjusts the training dataset to balance class distribution.
- It increases the representation of minority classes by first augmenting them with additional similar records.
- Records are repeated until their number matches either the majority class, or has increased 25-fold, whichever results in fewer repetitions.

FOLD 0

Time taken to generate 'balanced' training data: 6.03 seconds.

ORIGINAL TRAINING SET CLASS COUNTS:


TNRY_SEV
0           179304
1            43464
2             1935
Name: count, dtype: int64


UPSAMPLED TRAINING SET CLASS SIZES:


TNRY_SEV
0           179304
1           179304
2            48375
Name: count, dtype: int64


Class 0: 179304 records repeated 1 times, 
Class 1: 179304 records repeated 1 times, 
Class 2: 6615 records repeated 2 times, 11715 records repeated 3 times, 

Note that the validation set has NOT been altered.

VALIDATION SET CLASS COUNTS:


TNRY_SEV
0           44825
1           10867
2             484
Name: count, dtype: int64


Press enter to continue or enter c to cancel: c


We can now perform a partial grid search as we did in the case of simple oversampling. 

In [23]:
from models import custom_xgb_grid_search

instance: Type[model] = mtl_3sev_xgb_copy
balance_range: Union[List, np.array] = [1] + [10*i for i in range(1,10)]
max_depth_range: Union[List, np.array] =  [3,6]
n_estimators_range: Union[List, np.array] = [100,150]
results_filename: str = "mtl_3sev_augmented_oversample_gs"                           
timeout: Union[int, None] = None

custom_xgb_grid_search(instance=instance, 
                       balance_range=balance_range,
                       max_depth_range=max_depth_range,
                       n_estimators_range=n_estimators_range,
                       results_filename=results_filename,                           
                       timeout=timeout,)

Time taken: 56.0 minutes, 37.56 seconds.
Cumulative data for 40 of 40 grid points will now be stored in a .txt file.
File saved successfully to: F:\projects\road-safety\models\mtl_3sev_augmented_oversample_gs.txt


In [25]:
from models import grid_search_results

instance: Type[model] = mtl_3sev_xgb_copy
results_filename: str = "mtl_3sev_augmented_oversample_gs"
sort_by: Union[None, str] = "F*"

gs_results = grid_search_results(instance=instance,
                                 results_filename=results_filename,
                                 sort_by=sort_by)
display(gs_results)

,F*,precision 0,precision 1,precision 2,recall 0,recall 1,recall 2
"(70, 6, 150)",0.315959,0.891518,0.431323,0.109290,0.838754,0.490429,0.372671
"(70, 6, 100)",0.315959,0.891518,0.431323,0.109290,0.838754,0.490429,0.372671
"(70, 3, 150)",0.315959,0.891518,0.431323,0.109290,0.838754,0.490429,0.372671
"(70, 3, 100)",0.315959,0.891518,0.431323,0.109290,0.838754,0.490429,0.372671
"(80, 6, 150)",0.311654,0.892086,0.422307,0.094969,0.836880,0.466225,0.418219
"(80, 6, 100)",0.311654,0.892086,0.422307,0.094969,0.836880,0.466225,0.418219
"(80, 3, 150)",0.311654,0.892086,0.422307,0.094969,0.836880,0.466225,0.418219
"(80, 3, 100)",0.311654,0.892086,0.422307,0.094969,0.836880,0.466225,0.418219
"(90, 6, 150)",0.309609,0.891954,0.415607,0.085547,0.836836,0.442113,0.453416
"(90, 6, 100)",0.309609,0.891954,0.415607,0.085547,0.836836,0.442113,0.453416


The best result is slightly better than the best result we obtained by simple oversampling. The top four results are identical, so we'll use the grid point among them with the smallest ```max_depth``` and ```n_estimaters``` parameters to create a new instance of our ```model``` class, called ```mtl_3sev_augmented_oversample_xgb```. 

In [26]:
import re 
gs_params = gs_results.index[3]
coords = re.findall(r'\d+', gs_params)
balance, max_depth, n_estimators  = [int(coord) for coord in coords]

print_header(f"Augmented oversampling up to factor of {balance} with supplementary data, max_depth = {max_depth}, n_estimaters = {n_estimators}")

data: Type[process] = mtl_3sev_xgb_copy.data 
folds: Union[int, None] = mtl_3sev_xgb_copy.folds  
impute_strategy: Union[dict, None] = mtl_3sev_xgb_copy.impute_strategy
classifier: Union[DecisionTreeClassifier,
                  GradientBoostingClassifier,
                  LogisticRegression, 
                  RandomForestClassifier, 
                  XGBClassifier,] = XGBClassifier(objective='multi:softmax', # 'binary:logistic', if two classes
                                                  eval_metric='mlogloss',    # 'logloss', if two classes
                                                  max_delta_step=1,                                                   
                                                  importance_type='weight', 
                                                  max_depth = max_depth, 
                                                  n_estimators = n_estimators, 
                                                  nthread=-1,)
balance: Union[int, None] = balance
filename_stem: str = mtl_3sev_xgb_copy.filename_stem
model_dir: Path = mtl_3sev_xgb_copy.model_dir  
beta: Union[np.ndarray,None] = mtl_3sev_xgb_copy.beta
weights: Union[np.ndarray,None] = mtl_3sev_xgb_copy.weights

mtl_3sev_augmented_oversample_xgb = model(data=data,    
                                          folds=folds, 
                                          impute_strategy=impute_strategy, 
                                          classifier=classifier,
                                          balance=balance,
                                          filename_stem=filename_stem,
                                          model_dir=model_dir,
                                          beta=beta,
                                          weights=weights,)


AUGMENTED OVERSAMPLING UP TO FACTOR OF 70 WITH SUPPLEMENTARY DATA, MAX_DEPTH = 3, N_ESTIMATERS = 100


SEPARATING FEATURES FROM TARGETS

self.X_train/self.X_test, self.y_train/self.y_test

MAPPING ORDINAL FEATURE/TARGET CODES

NUM_VEH: {1: 0, 2: 1, 9: 2}
LIGHT: {1: 3, 2: 2, 3: 1, 4: 0}
SPD_LIM: {'<50': 0, 50: 5, 60: 6, 70: 7, 80: 8, 90: 9, 100: 10}
TNRY_SEV: {0: 0, 1: 1, 2: 2}

MAPPING CATEGORICAL FEATURE/TARGET CODES

WKDY_WKND: {'WKDY': 0, 'WKND': 1}
PED: {'N': 0, 'Y': 1}
MTRCYC: {'N': 0, 'Y': 1}
ACCDN_TYPE: {'vehicle': 0, 'pedestrian': 1, 'cyclist': 2, 'animal': 3, 'fixed object': 4, 'no collision': 5, 'other': 6}
BICYC: {'N': 0, 'Y': 1}
HOUR: {'00:00:00-03:59:00': 0, '04:00:00-07:59:00': 1, '08:00:00-11:59:00': 2, '12:00:00-15:59:00': 3, '16:00:00-19:59:00': 4, '20:00:00-23:59:00': 5}
LT_TRK: {'N': 0, 'Y': 1}
RD_CONFG: {1: 0, 23: 1, 45: 2, 9: 3}
ASPECT: {'Straight': 0, 'Curve': 1}
LONG_LOC: {12: 0, 33: 1, 34: 2, 40: 3, 69: 4, 99: 5}
ZONE: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 9: 6}

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal',
                                                  OrdinalEncoder(categories=[[0,
                                                                              1,
                                                                              2],
                                                                             [0,
                                                                              1,
                                                                              2,
                                                                              3],
                                                                             [0,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9,
                                                                              10]],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=nan),
                                                  ['NUM_VEH', 'LIGHT',
                                                   'SPD_LIM']),
                                                 ('onehot',
                                                  OneHotEncoder(categories=[[0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type='weight',
                               interaction_constraints=None, learning_rate=None,
                 


       MAKING PREDICTIONS ON VALIDATION FOLD

CV 1:  FOLDING, BALANCING WITH SUPPLEMENTARY DATA, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 2:  FOLDING, BALANCING WITH SUPPLEMENTARY DATA, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 3:  FOLDING, BALANCING WITH SUPPLEMENTARY DATA, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 4:  FOLDING, BALANCING WITH SUPPLEMENTARY DATA, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

Target, prediction, and probabilities for each validation fold stored in self.preds dictionary.

Feature importances corresponding to each validation fold stored in self.feature_importances dictionary.

SAVING PIPELINE

F:\projects\road-safety\models\mtl_3sev_augmented_oversample_xgb.joblib

Metrics (including confusion matrices) contained in dictionary self.eval
Metrics (excluding confusion matrices) also contained in dataframe self.eval_df

OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DO

,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.759951,0.565100,0.469522,0.565100,0.488540,0.517944,0.337823,0.804338
1,0.760129,0.565197,0.469785,0.565197,0.489305,0.518878,0.335327,0.803376
2,0.757868,0.552462,0.464051,0.552462,0.480861,0.507823,0.334148,0.807631
3,0.763226,0.563961,0.471457,0.563961,0.490218,0.518822,0.343158,0.805648
4,0.761104,0.570727,0.470760,0.570727,0.490279,0.520856,0.337233,0.805839
mean,0.760456,0.563490,0.469115,0.563490,0.487840,0.516865,0.337538,0.805366



CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


Feature importances corresponding to validation set(s) contained in self.feature_importances and self.feature_importances_df.


FEATURE IMPORTANCE



interactive(children=(Dropdown(description='Sort by:', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_clas…

As with simple oversampling, augmented oversampling has boosted performance of our model to a still unsatisfactory level, slightly worse than what we obtained by merely adjusting thresholds. 

<a id='combining'></a>
## Combining threshold adjustment and oversampling
↑↑ [Contents](#contents) ↑ [Augmented oversampling](#augmented_oversampling) ↓ [Final Model](#final_model)

For completeness, we try combining threshold adjustments with oversampling. We are unable to obtain a signficantly better $F^*$ score.

In [22]:
from typing import Tuple
from models import threshold_grid_search

instance: Type[model] = mtl_3sev_augmented_oversample_xgb

priority: List[Tuple[int, str]] = [
                                   (2,'help'), 
                                   (1,'help'), 
                                   # (0,'hinder'),
                                  ]

threshold: np.ndarray = np.array([
                                  np.arange(0.05, 0.95, 0.05), 
                                  np.arange(0.05, 0.95, 0.05), 
                                  # np.arange(0.30, 0.7, 0.05),
                                 ])

timeout: Union[int, None] = None # Integer n means incomplete grid search will stop shortly after n seconds.

threshold_grid_search(instance=instance,
                      priority=priority,
                      threshold=threshold,
                      timeout=timeout,);

Best mean F* score so far (F* averaged over validation folds): 0.10420356359825922
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.05}), hinder OrderedDict()
Time taken: 0.47 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.11984441632077225
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.1}), hinder OrderedDict()
Time taken: 0.94 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.12376621886535695
Corresponding thresholds: help OrderedDict({2: 0.05, 1: 0.15}), hinder OrderedDict()
Time taken: 1.37 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.14663469163159806
Corresponding thresholds: help OrderedDict({2: 0.1, 1: 0.05}), hinder OrderedDict()
Time taken: 8.73 seconds and counting

Best mean F* score so far (F* averaged over validation folds): 0.16463237060076205
Corresponding thresholds: help OrderedDict({2: 0.1, 1: 0.1}), hinder OrderedDict()
Time tak

<a id='final_model'></a>
## Final model
↑↑ [Contents](#contents) ↑ [Combining threshold adjustment and oversampling](#combining) ↓ [Evaluation](#evaluation)

Our original model, with threshold adjustments, gave best performance with respect to our $F^*$ metric. This will be our final model. 

In [23]:
data: Type[process] = mtl_3sev    
folds: Union[int, None] = 5 
impute_strategy: Union[dict, None] = None 
classifier: Union[DecisionTreeClassifier,
                  GradientBoostingClassifier,
                  LogisticRegression, 
                  RandomForestClassifier, 
                  XGBClassifier,] = XGBClassifier(objective='multi:softmax', 
                                                  eval_metric='mlogloss',    
                                                  max_delta_step=1,                                                   
                                                  importance_type='weight', 
                                                  max_depth = 3, 
                                                  n_estimators = 100, 
                                                  nthread=-1,)
balance: Union[int, None] = None
filename_stem: str = "mtl_3sev_final"
model_dir: Path = path["models"]   
beta: Union[np.ndarray,None] = np.array([0.5, 1, 2])
weights: Union[np.ndarray,None] = np.array([1,4,16])
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([(2,0.05), (1,0.05)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = OrderedDict([(0,0.75)]) 

mtl_3sev_final = model(data=data,    
                       folds=folds, 
                       impute_strategy=impute_strategy, 
                       classifier=classifier,
                       balance=balance,
                       filename_stem=filename_stem,
                       model_dir=model_dir,
                       beta=beta,
                       weights=weights,
                       threshold_dict_help=threshold_dict_help,
                       threshold_dict_hinder=threshold_dict_hinder,)


SEPARATING FEATURES FROM TARGETS

self.X_train/self.X_test, self.y_train/self.y_test

MAPPING ORDINAL FEATURE/TARGET CODES

NUM_VEH: {1: 0, 2: 1, 9: 2}
LIGHT: {1: 3, 2: 2, 3: 1, 4: 0}
SPD_LIM: {'<50': 0, 50: 5, 60: 6, 70: 7, 80: 8, 90: 9, 100: 10}
TNRY_SEV: {0: 0, 1: 1, 2: 2}

MAPPING CATEGORICAL FEATURE/TARGET CODES

WKDY_WKND: {'WKDY': 0, 'WKND': 1}
PED: {'N': 0, 'Y': 1}
MTRCYC: {'N': 0, 'Y': 1}
ACCDN_TYPE: {'vehicle': 0, 'pedestrian': 1, 'cyclist': 2, 'animal': 3, 'fixed object': 4, 'no collision': 5, 'other': 6}
BICYC: {'N': 0, 'Y': 1}
HOUR: {'00:00:00-03:59:00': 0, '04:00:00-07:59:00': 1, '08:00:00-11:59:00': 2, '12:00:00-15:59:00': 3, '16:00:00-19:59:00': 4, '20:00:00-23:59:00': 5}
LT_TRK: {'N': 0, 'Y': 1}
RD_CONFG: {1: 0, 23: 1, 45: 2, 9: 3}
ASPECT: {'Straight': 0, 'Curve': 1}
LONG_LOC: {12: 0, 33: 1, 34: 2, 40: 3, 69: 4, 99: 5}
ZONE: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 9: 6}
RD_COND: {11: 0, 12: 1, 13: 2, 14: 3, 15: 4, 16: 5, 17: 6, 18: 7, 19: 8, 20: 9, 99: 10}
RDWX: {'N': 0,

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal',
                                                  OrdinalEncoder(categories=[[0,
                                                                              1,
                                                                              2],
                                                                             [0,
                                                                              1,
                                                                              2,
                                                                              3],
                                                                             [0,
                                                                              5,
                                                                              6,
                                                                              7,
                                                                              8,
                                                                              9,
                                                                              10]],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=nan),
                                                  ['NUM_VEH', 'LIGHT',
                                                   'SPD_LIM']),
                                                 ('onehot',
                                                  OneHotEncoder(categories=[[0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5],
                                                                            [0,
                                                                             1],
                                                                            [0,
                                                                             1,
                                                                             2,...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type='weight',
                               interaction_constraints=None, learning_rate=None,
                 


       MAKING PREDICTIONS ON VALIDATION FOLD

CV 1:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 2:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 3:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 4:  FOLDING, FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

Target, prediction, and probabilities for each validation fold stored in self.preds dictionary.

Feature importances corresponding to each validation fold stored in self.feature_importances dictionary.

SAVING PIPELINE

F:\projects\road-safety\models\mtl_3sev_final_xgb.joblib

Metrics (including confusion matrices) contained in dictionary self.eval
Metrics (excluding confusion matrices) also contained in dataframe self.eval_df

OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.855419,0.447646,0.554333,0.447646,0.471138,0.453246,0.483741,0.839987
1,0.854511,0.447479,0.753535,0.447479,0.472709,0.453407,0.479506,0.841437
2,0.855917,0.448540,0.639252,0.448540,0.472910,0.454347,0.485859,0.842870
3,0.855525,0.449049,0.723032,0.449049,0.475320,0.455247,0.484017,0.840842
4,0.855523,0.448472,0.649905,0.448472,0.473287,0.454369,0.484091,0.838514
mean,0.855379,0.448237,0.664011,0.448237,0.473073,0.454123,0.483443,0.840730



CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



...

Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.379237,0.513457,0.428377,0.513457,0.306518,0.364093,0.156122,0.839987
1,0.379450,0.521721,0.431351,0.521721,0.310259,0.370099,0.159289,0.841437
2,0.380643,0.512942,0.427142,0.512942,0.306176,0.362889,0.154420,0.842870
3,0.378898,0.519385,0.430505,0.519385,0.309112,0.368392,0.157626,0.840842
4,0.377713,0.529587,0.431544,0.529587,0.311393,0.373073,0.155697,0.838514
mean,0.379188,0.519418,0.429784,0.519418,0.308692,0.367709,0.156631,0.840730



Adjusted predictions stored in self.adjusted_preds


Our model can make predictions on any dataset that matches the required format and feature set of our training or validation data, as demonstrated in the code cell below.

In [27]:
from models import inference
import random 

instance: Type[model] = mtl_3sev_final

random_data = dict.fromkeys(instance.data.features)
for feature in random_data.keys():
    random_data[feature] = random.choice(list(instance.class_codes[feature].values()))

random_data_df = pd.DataFrame(random_data, columns = instance.data.features, index=[0])
    
X: Union[None, pd.DataFrame] = random_data_df
y: Union[None, pd.DataFrame] = None

print("Output shows probabilities for each class, prediction corresponding to maximum probability, prediction based on adjusted thresholds.")
random_data_pred = inference(instance=instance, 
                                X=X, 
                                y=y,)
display(random_data_pred)

Output shows probabilities for each class, prediction corresponding to maximum probability, prediction based on adjusted thresholds.


,MONTH,HOUR,WKDY_WKND,NUM_VEH,SPD_LIM,ACCDN_TYPE,RD_COND,LIGHT,ZONE,PUB_PRIV_RD,...,LT_TRK,HVY_VEH,MTRCYC,BICYC,PED,prob 0,prob 1,prob 2,pred,adj pred
0,2,5,0,1,5,5,6,0,3,0,...,1,1,1,1,0,0.136091,0.6164,0.247508,1,2


<a id='evaluation'></a>
## Evaluation
↑↑ [Contents](#contents) ↑ [Final model](#final_model) ↓ [Other classifiers](#other_classifiers)

Now for the moment of truth. We evaluate our model on the test data, which has been 'locked in the vault' until this point. Just like with the training data, we've applied the same mappings to the test data but have not manipulated it further. The confusion matrices below show results similar to those observed during our cross-validation process.

In [354]:
from models import infer_eval

instance: Type[model] = mtl_3sev_final
X: Union[None, pd.DataFrame] = instance.X_test
y: Union[None, pd.DataFrame] = instance.y_test
verbose: Union[None, List[str]] = ['info', 'adjusted_eval',]

infer_eval(instance=instance,
           X=X, 
           y=y, 
           verbose=verbose,);

Using in-memory model.

MAKING PREDICTIONS ON GIVEN DATA


THRESHOLD-ADJUSTED PREDICTIONS: 'HELP' ORDEREDDICT({2: 0.05, 1: 0.05}), 'HINDER' ORDEREDDICT({0: 0.75})


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Output()


OTHER METRICS



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.379849,0.515272,0.429691,0.515272,0.308499,0.366807,0.158275,0.838454


<a id='other_classifiers'></a>
## Other classifiers
↑↑ [Contents](#contents) ↑ [Evaluation](#evaluation) ↓ [Conclusion](#conclusion)

Let's try logistic regression. For logistic regression, we need to impute missing values. The LogisticRegression classifier is somewhat slower than XGBoost, and its performance is slightly worse.

In [26]:
data: Type[process] = mtl_3sev    
folds: Union[int, None] = 5 
impute_strategy: Union[dict, None] = {'categorical': 'most_frequent', 'ordinal': 'most_frequent', 'constant' : -1}
classifier: Union[DecisionTreeClassifier,
                  GradientBoostingClassifier,
                  LogisticRegression, 
                  RandomForestClassifier, 
                  XGBClassifier,] = LogisticRegression(max_iter=1000, n_jobs=-1)
balance: Union[int, None] = None 
filename_stem: str = "mtl_3sev"
model_dir: Path = path["models"]   
beta: Union[np.ndarray,None] = np.array([0.5, 1, 2])
weights: Union[np.ndarray,None] = np.array([1,4,16])
threshold_dict_help: Union[None, OrderedDict[Union[int, str], float]] = None     
threshold_dict_hinder: Union[None, OrderedDict[Union[int, str], float]] = None   

mtl_3sev_lr = model(data=data,    
                    folds=folds, 
                    impute_strategy=impute_strategy, 
                    classifier=classifier,
                    balance=balance,
                    filename_stem=filename_stem,
                    model_dir=model_dir,
                    beta=beta,
                    weights=weights,
                    threshold_dict_help=threshold_dict_help,
                    threshold_dict_hinder=threshold_dict_hinder,)


SEPARATING FEATURES FROM TARGETS

self.X_train/self.X_test, self.y_train/self.y_test

MAPPING ORDINAL FEATURE/TARGET CODES

LIGHT: {1: 3, 2: 2, 3: 1, 4: 0}
SPD_LIM: {'<50': 0, 50: 5, 60: 6, 70: 7, 80: 8, 90: 9, 100: 10}
NUM_VEH: {1: 0, 2: 1, 9: 2}
TNRY_SEV: {0: 0, 1: 1, 2: 2}

MAPPING CATEGORICAL FEATURE/TARGET CODES

PUB_PRIV_RD: {1: 0, 2: 1}
MTRCYC: {'N': 0, 'Y': 1}
ASPECT: {'Straight': 0, 'Curve': 1}
MONTH: {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
ACCDN_TYPE: {'vehicle': 0, 'pedestrian': 1, 'cyclist': 2, 'animal': 3, 'fixed object': 4, 'no collision': 5, 'other': 6}
LONG_LOC: {12: 0, 33: 1, 34: 2, 40: 3, 69: 4, 99: 5}
WKDY_WKND: {'WKDY': 0, 'WKND': 1}
HVY_VEH: {'N': 0, 'Y': 1}
RDWX: {'N': 0, 'Y': 1}
RD_CONFG: {1: 0, 23: 1, 45: 2, 9: 3}
ZONE: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 9: 6}
WEATHER: {11: 0, 12: 1, 13: 2, 14: 3, 15: 4, 16: 5, 17: 6, 18: 7, 19: 8, 99: 9}
HOUR: {'00:00:00-03:59:00': 0, '04:00:00-07:59:00': 1, '08:00:00-11:59:00': 2, '12:

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal_imputer',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='most_frequent'),
                                                  ['LIGHT', 'SPD_LIM',
                                                   'NUM_VEH']),
                                                 ('categorical_imputer',
                                                  SimpleImputer(fill_value=-1,
                                                                strategy='most_frequent'),
                                                  ['PUB_PRIV_RD', 'MTRCYC',
                                                   'ASPECT', 'MONTH',
                                                   'ACCDN_TYPE', 'LONG_LOC',
                                                   'WKDY_WKND', 'HVY...
                                                                             3],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6,
                                                                             7,
                                                                             8,
                                                                             9],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5],
                                                                            [0,
                                                                             1,
                                                                             2,
                                                                             3,
                                                                             4,
                                                                             5,
                                                                             6,
                                                                             7,
                                                                             8,
                                                                             9,
                                                                             10],
                                                                            [0,
                                                                             1],
                            


       MAKING PREDICTIONS ON VALIDATION FOLD

CV 1:  IMPUTING FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 2:  IMPUTING FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 3:  IMPUTING FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

CV 4:  IMPUTING FITTING

       MAKING PREDICTIONS ON VALIDATION FOLD

Target, prediction, and probabilities for each validation fold stored in self.preds dictionary.

Feature importances corresponding to each validation fold stored in self.feature_importances dictionary.

SAVING PIPELINE

F:\projects\road-safety\models\mtl_3sev_lr.joblib

Metrics (including confusion matrices) contained in dictionary self.eval
Metrics (excluding confusion matrices) also contained in dataframe self.eval_df

OTHER METRICS (ROWS CORRESPOND TO VALIDATION FOLDS IF CROSS-VALIDATION HAS BEEN DONE)



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.850790,0.441793,0.713440,0.441793,0.464591,0.446988,0.462802,0.803564
1,0.850666,0.440110,0.820250,0.440110,0.462086,0.445033,0.461982,0.804719
2,0.852570,0.443067,0.550925,0.443067,0.465812,0.448290,0.470700,0.805905
3,0.851769,0.441338,0.549749,0.441338,0.463756,0.446396,0.466937,0.803139
4,0.851304,0.443069,0.690953,0.443069,0.467311,0.448601,0.464948,0.802856
mean,0.851420,0.441876,0.665064,0.441876,0.464711,0.447062,0.465474,0.804036



CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Dropdown(options=(0, 1, 2, 3, 4), value=0)

Output()


Feature importances corresponding to validation set(s) contained in self.feature_importances and self.feature_importances_df.


FEATURE IMPORTANCE



interactive(children=(Dropdown(description='Sort by:', options=(0, 1, 2, 3, 4), value=0), Output()), _dom_clas…

It is simply a matter of replacing the ```classifier``` attribute with ```RandomForestClassifier```,  ```DecisionTreeClassifier```, or ```GradientBoostingClassifier``` to try out these classifiers. Similarly, we can slice and dice the data in various ways, and try to predict other variables, by modifying the attributes of the ```process``` class.

<a id='analysis'></a>
## Retrospective analysis
↑↑ [Contents](#contents) ↑ [Other classifiers](#other_classifiers) ↓ [Conclusion](#conclusion)

Our final (and best) model performs poorly on both training data and validation data.

In [29]:
# Run this cell once and the next cell up to k times, where k = instance.folds as in k-fold cross validation
instance: Type[model] = mtl_3sev_final
generate_folds = instance.kfold.split(instance.X_train, instance.y_train)
fold = -1

In [30]:
# Run this cell to sequentially generate train/val indices for validation folds.
try:
    train_idx, val_idx = next(generate_folds)
    fold += 1
except StopIteration:
    print(f"No more folds. Rerun previous code cell to start over.")

In [337]:
from models import infer_eval

instance: Type[model] = mtl_3sev_final
X: Union[None, pd.DataFrame] = instance.X_train.iloc[train_idx]
y: Union[None, pd.DataFrame] = instance.y_train.iloc[train_idx]
verbose: Union[None, List[str]] = ['adjusted_eval'] # 'unadjusted_eval' for predictions corresponding to max probability

print_header(f"Fold {fold}")

print_header("Training set")
infer_eval(instance=instance,
           X=X, 
           y=y, 
           verbose=verbose,);

X: Union[None, pd.DataFrame] = instance.X_train.iloc[val_idx]
y: Union[None, pd.DataFrame] = instance.y_train.iloc[val_idx]

print_header("Validation set")
infer_eval(instance=instance,
           X=X, 
           y=y, 
           verbose=verbose,);


FOLD 0


TRAINING SET


THRESHOLD-ADJUSTED PREDICTIONS: 'HELP' ORDEREDDICT({2: 0.05, 1: 0.05}), 'HINDER' ORDEREDDICT({0: 0.75})


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Output()


OTHER METRICS



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.379964,0.53193,0.432711,0.53193,0.313275,0.375168,0.159085,0.842784



VALIDATION SET


THRESHOLD-ADJUSTED PREDICTIONS: 'HELP' ORDEREDDICT({2: 0.05, 1: 0.05}), 'HINDER' ORDEREDDICT({0: 0.75})


CONFUSION MATRIX: WEIGHTED AVERAGE CLASS-WISE F_BETA SCORES AT BOTTOM RIGHT (BETA VALUES {0: 0.5, 1: 1.0, 2: 2.0}, WEIGHTS {0: 1.0, 1: 4.0, 2: 16.0})



Output()


OTHER METRICS



,ACC,BACC,precision,recall,F1,F2,MCC,ROC-AUC
0,0.381408,0.528788,0.432509,0.528788,0.3129,0.373972,0.161723,0.842269


The model exhibits clear signs of underfitting, with poor performance rooted in the inadequate discriminative power of the features. These features lack the ability to provide sufficient separation between different classes or outcomes, which is essential for effective classification. As a result, the model performs consistently poorly on both training and validation data, indicating that even with optimal tuning, the model would be unable to achieve satisfactory performance due to limitations within the dataset itself.

The low discriminative value of these features is evident from the mutual information scores, all of which are notably low. For instance, 'PED' unsurprisingly has one of the highest mutual information scores, as an accident involving a pedestrian will necessarily fall into Class 1 or 2 rather than Class 0. However, even this feature's mutual information score remains low, underscoring the dataset's overall inadequacy in capturing meaningful class distinctions.

In [338]:
from sklearn.feature_selection import mutual_info_classif
import time

tic = time.time()
mi_scores = mutual_info_classif(X.fillna(-1), np.ravel(y))
toc = time.time()

mi_series = pd.Series(data=mi_scores, index=X.columns)
mi_series = mi_series.sort_values(ascending=False)

print_header(f"Fold {fold}")
print("Mutual information scores (training set)\n".upper())
print(mi_series)
print(f"\nTime taken: {toc - tic:.2f} seconds.")


FOLD 0

MUTUAL INFORMATION SCORES (TRAINING SET)

ACCDN_TYPE     0.081321
PED            0.068367
LONG_LOC       0.035185
RD_CONFG       0.024205
BICYC          0.022583
NUM_VEH        0.022506
SPD_LIM        0.017097
LIGHT          0.014252
PUB_PRIV_RD    0.012737
HOUR           0.011417
WEATHER        0.010875
RD_COND        0.010018
ZONE           0.009790
RDWX           0.008203
ASPECT         0.008101
LT_TRK         0.006209
MTRCYC         0.005754
HVY_VEH        0.003108
MONTH          0.002972
WKDY_WKND      0.002583
dtype: float64

Time taken: 7.02 seconds.


The dataset is sparse, with each unique feature combination appearing only 1.5 times on average, which could present a challenge for model learning. However, the primary issue is not the sparsity itself but rather the overlap in feature space among clusters of similar feature combinations. Even within these clusters, a significant proportion of records fall into multiple classes, complicating the model's ability to generalize effectively. This overlap, with inconsistent class labels among closely related feature values, suggests that the features lack the discriminative power needed for reliable classification, more fundamentally limiting the model's performance.

In [32]:
from retrospective import feature_combination_analysis

instance: Type[model] = mtl_3sev_final
train_idx: np.ndarray = train_idx
val_idx: np.ndarray = val_idx
features: Union[None, List] = None
ignore: Union[None, List] = ['RDWX']
two_missing_values_considered_equal: bool = True
sentinel: int = -1

print_header(f"Fold {fold}")

print("We ignore 'RDWX' because a small minority of values are 1 ('Y') while the rest are missing."
       "\nThis is in spite of the fact that possible values are supposed to include 0 ('N')." 
       "\nWe suspect that absence of 'Y' means 'N' in most cases, but as we cannot be certain in all cases, we simply disregard this feature here.\n")

feature_combination_analysis(instance=instance,
                             train_idx=train_idx,
                             val_idx=val_idx,
                             features=features,
                             ignore=ignore,
                             two_missing_values_considered_equal=two_missing_values_considered_equal,
                             sentinel=sentinel,)


FOLD 0

We ignore 'RDWX' because a small minority of values are 1 ('Y') while the rest are missing.
This is in spite of the fact that possible values are supposed to include 0 ('N').
We suspect that absence of 'Y' means 'N' in most cases, but as we cannot be certain in all cases, we simply disregard this feature here.

- Ignoring: ['RDWX']
- Replacing NaN values by -1 so that two missing values are considered the same.
- Two feature vectors are considered identical if they agree in all coordinates except, possibly, those corresponding to ignored feature(s).
- Otherwise, two feature vectors are considered distinct.
- Table rows...: 'train + val' all records, whether from training set or validation set
                 'train' records from training set only
                 'val' records from validation set only
                 'val seen' validation set records with feature combinations that also appear in the training set
                 'val unseen' validation set records with featu

,,all,distinct,ratio
0,train + val,"280,879","182,080",1.5426
1,train,"224,703","151,256",1.4856
2,val,"56,176","45,810",1.2263
3,val unseen,"31,665","30,824",1.0273
4,val seen,"24,511","14,986",1.6356
5,seen/unseen,0.7741,0.4862,-



CLASS 0


,,all,distinct,ratio
0,train + val,"224,129","149,580",1.4984
1,train,"179,304","124,094",1.4449
2,val,"44,825","37,270",1.2027
3,val unseen,"26,156","25,486",1.0263
4,val seen,"18,669","11,784",1.5843
5,seen/unseen,0.7138,0.4624,-



CLASS 1


,,all,distinct,ratio
0,train + val,"54,331","40,906",1.3282
1,train,"43,464","33,716",1.2891
2,val,"10,867","9,688",1.1217
3,val unseen,"7,326","7,190",1.0189
4,val seen,"3,541","2,498",1.4175
5,seen/unseen,0.4833,0.3474,-



CLASS 2


,,all,distinct,ratio
0,train + val,"2,419","2,361",1.0246
1,train,"1,935","1,893",1.0222
2,val,484,482,1.0041
3,val unseen,470,468,1.0043
4,val seen,14,14,1
5,seen/unseen,0.0298,0.0299,-


In [33]:
from retrospective import extreme_examples

instance: Type[model] = mtl_3sev_final
train_idx: np.ndarray = train_idx
val_idx: np.ndarray = val_idx
features: Union[None, list] = None
ignore: Union[None, list] = ['RDWX']
drop_row_if_more_than_this_many_missing_values: Union[None, list] = 0
two_missing_values_considered_equal: bool = True
sentinel: int = -1
remove_row_if_more_than_this_many_missing_values: Union[None, int] = None
distinct_classes: int = 3
high_to_low_multiplicity: bool = True

print_header(f"Fold {fold}")

extreme_examples(instance=instance,
                 train_idx=train_idx,
                 val_idx=val_idx,
                 features=features,
                 ignore=ignore,
                 drop_row_if_more_than_this_many_missing_values=drop_row_if_more_than_this_many_missing_values,
                 two_missing_values_considered_equal=two_missing_values_considered_equal,
                 sentinel=sentinel,
                 distinct_classes=distinct_classes,
                 high_to_low_multiplicity=high_to_low_multiplicity,)


FOLD 0

- The feature combination



WKDY_WKND,LIGHT,SPD_LIM,ACCDN_TYPE,BICYC,RD_CONFG,ASPECT,ZONE,RD_COND,MONTH,PUB_PRIV_RD,PED,MTRCYC,HOUR,LT_TRK,NUM_VEH,LONG_LOC,WEATHER,HVY_VEH
0,3,5,0,0,1,0,2,0,5,0,0,0,3,1,1,0,0,0



  appears 107 times in training, and 20 times in validation.

- For this specific feature combination, in the training set, the class distribution is



TNRY_SEV,count,%
0,70,65.420561
1,37,34.579439



- A reasonable model might assign probabilities to each class according to this distribution.

- For the same feature combination, in the validation set, the class distribution is 



TNRY_SEV,count,%
0,12,60.000000
1,7,35.000000
2,1,5.000000



- NO MODEL CAN CORRECTLY CLASSIFY MORE THAN ~65% (RESP. ~60%) OF THIS TRAINING (RESP. VALIDATION) SAMPLE.

- Our classifier gives the following probabilities and prediction(s):



prob 0,prob 1,prob 2,pred,adj pred
0.669663,0.325913,0.004424,0,1



- Prediction based on maximum probability correctly classifies 60.00% of this validation sample.

- Prediction based on adjusted thresholds correctly classifies 35.00% of this validation sample.



Press enter to continue or enter c to cancel: c


In [34]:
from retrospective import extreme_examples

distinct_classes: int = 2
high_to_low_multiplicity: bool = True

print_header(f"Fold {fold}")

extreme_examples(instance=instance,
                 train_idx=train_idx,
                 val_idx=val_idx,
                 features=features,
                 ignore=ignore,
                 drop_row_if_more_than_this_many_missing_values=drop_row_if_more_than_this_many_missing_values,
                 two_missing_values_considered_equal=two_missing_values_considered_equal,
                 sentinel=sentinel,
                 distinct_classes=distinct_classes,
                 high_to_low_multiplicity=high_to_low_multiplicity,)


FOLD 0

- The feature combination



WKDY_WKND,LIGHT,SPD_LIM,ACCDN_TYPE,BICYC,RD_CONFG,ASPECT,ZONE,RD_COND,MONTH,PUB_PRIV_RD,PED,MTRCYC,HOUR,LT_TRK,NUM_VEH,LONG_LOC,WEATHER,HVY_VEH
0,3,5,0,0,1,0,2,0,8,0,0,0,3,1,1,0,0,0



  appears 108 times in training, and 32 times in validation.

- For this specific feature combination, in the training set, the class distribution is



TNRY_SEV,count,%
0,65,60.185185
1,42,38.888889
2,1,0.925926



- A reasonable model might assign probabilities to each class according to this distribution.

- For the same feature combination, in the validation set, the class distribution is 



TNRY_SEV,count,%
0,22,68.750000
1,10,31.250000



- NO MODEL CAN CORRECTLY CLASSIFY MORE THAN ~60% (RESP. ~68%) OF THIS TRAINING (RESP. VALIDATION) SAMPLE.

- Our classifier gives the following probabilities and prediction(s):



prob 0,prob 1,prob 2,pred,adj pred
0.661401,0.334350,0.004249,0,1



- Prediction based on maximum probability correctly classifies 68.75% of this validation sample.

- Prediction based on adjusted thresholds correctly classifies 31.25% of this validation sample.



Press enter to continue or enter c to cancel: c


In [35]:
distinct_classes: int = 1
high_to_low_multiplicity: bool = True

print_header(f"Fold {fold}")

extreme_examples(instance=instance,
                 train_idx=train_idx,
                 val_idx=val_idx,
                 features=features,
                 ignore=ignore,
                 drop_row_if_more_than_this_many_missing_values=drop_row_if_more_than_this_many_missing_values,
                 two_missing_values_considered_equal=two_missing_values_considered_equal,
                 sentinel=sentinel,
                 distinct_classes=distinct_classes,
                 high_to_low_multiplicity=high_to_low_multiplicity,)


FOLD 0

- The feature combination



WKDY_WKND,LIGHT,SPD_LIM,ACCDN_TYPE,BICYC,RD_CONFG,ASPECT,ZONE,RD_COND,MONTH,PUB_PRIV_RD,PED,MTRCYC,HOUR,LT_TRK,NUM_VEH,LONG_LOC,WEATHER,HVY_VEH
0,3,5,0,0,1,0,2,0,3,0,0,0,3,1,1,1,0,0



  appears 27 times in training, and 12 times in validation.

- For this specific feature combination, in the training set, the class distribution is



TNRY_SEV,count,%
0,24,88.888889
1,3,11.111111



- A reasonable model might assign probabilities to each class according to this distribution.

- For the same feature combination, in the validation set, the class distribution is 



TNRY_SEV,count,%
0,12,100.000000



- NO MODEL CAN CORRECTLY CLASSIFY MORE THAN ~88% OF THIS TRAINING SAMPLE.

- Our classifier gives the following probabilities and prediction(s):



prob 0,prob 1,prob 2,pred,adj pred
0.842686,0.155228,0.002086,0,1



- Prediction based on maximum probability correctly classifies 100.00% of this validation sample.

- Prediction based on adjusted thresholds correctly classifies 0.00% of this validation sample.



Press enter to continue or enter c to cancel: c


In [72]:
distinct_classes: int = 1
high_to_low_multiplicity: bool = False

print_header(f"Fold {fold}")

extreme_examples(instance=instance,
                 train_idx=train_idx,
                 val_idx=val_idx,
                 features=features,
                 ignore=ignore,
                 drop_row_if_more_than_this_many_missing_values=drop_row_if_more_than_this_many_missing_values,
                 two_missing_values_considered_equal=two_missing_values_considered_equal,
                 sentinel=sentinel,
                 distinct_classes=distinct_classes,
                 high_to_low_multiplicity=high_to_low_multiplicity,)

- The feature combination



WKDY_WKND,LIGHT,SPD_LIM,ACCDN_TYPE,BICYC,RD_CONFG,ASPECT,ZONE,RD_COND,MONTH,PUB_PRIV_RD,PED,MTRCYC,HOUR,LT_TRK,NUM_VEH,LONG_LOC,WEATHER,HVY_VEH
0,1,0,0,0,0,0,1,0,1,0,0,0,1,1,1,1,0,0



  appears 1 times in training, and 1 times in validation.

- For this specific feature combination, in the training set, the class distribution is



TNRY_SEV,count,%
0,1,100.000000



- A model might not be able to assign meaningful probabilities to each class, unless there are 'similar' feature combinations in the training data.

- For the same feature combination, in the validation set, the class distribution is 



...

prob 0,prob 1,prob 2,pred,adj pred
0.957103,0.041829,0.001067,0,0



- Prediction based on maximum probability correctly classifies 100.00% of this validation sample.

- Prediction based on adjusted thresholds correctly classifies 100.00% of this validation sample.

SIMILAR FEATURE COMBINATIONS

- Training set contains 14 records that agree with the above feature combination on 18 features.

- Over these similar feature combinations, in the training set, the class distribution is



TNRY_SEV,count,%
0,13,92.857143
1,1,7.142857



- Does this reflect the probabilities given by the classifer? Should it?



Press enter to continue or enter c to cancel: c


Exacerbating the class imbalance, we observe that while most records in the training set have a "nearest neighbor" differing by only 0 or 1 feature, records in the minority class tend to be more isolated. This isolation means that minority class instances are less likely to have close neighbors, further limiting the model's ability to learn generalizable patterns for this class. Consequently, this structural sparsity within the minority class contributes to its under-representation in the model's predictions, compounding the effects of class imbalance.

In [343]:
from retrospective import min_distance_approx, combined_value_counts
import time

print_header(f"Fold {fold}")

df: pd.DataFrame = instance.X_train.iloc[train_idx].copy().fillna(-1)
filename: str = f"min_distances_fold_{fold}.json"
save_as: Path = instance.model_dir.joinpath(filename)

try:
    min_distances = pd.read_json(save_as, typ='series')
except FileNotFoundError:
    print("Generating distances for the first time.")
    tic = time.time()
    min_distances = min_distance_approx(df=df)
    toc = time.time()
    print(f"\nTime taken: {toc - tic} seconds.")
    print("\nSaving pd.series to json file {save_as}.\n")
    df.to_json(save_as, index=True)
print("Value d at index idx means:\n")
print("- There exists at least one record in df that differs from df.loc[idx] in exactly d features, and")
print("- No other record in df differs from df.loc[idx] in fewer than d features.\n")
print("Note: Some values may be incorrect as we're using an approximate nearest neighbors approach for efficiency.")
print("      This may occasionally miss the exact closest match in large or high-dimensional datasets.\n")

print(min_distances)
print("\nHere are the value counts:\n")
min_dist_vc = min_distances.value_counts()
min_distances_df = min_distances.to_frame(name='nn dist')
display(combined_value_counts(df=min_distances_df,column='nn dist'))

max_min_dist = min_dist_vc.index.max()
number_isolated = min_dist_vc.loc[max_min_dist]
print(f"\nE.g. there are {number_isolated} records in df that differ from every other record in df in at least {max_min_dist} features.")

isolated_record = df.loc[min_distances[min_distances == max_min_dist].index[0]]
isolated_NN = similar_feature_combinations(df=df,
                             feature_combination=isolated_record,
                             exact_matches=None,
                             total_matches=len(feature_combination) - max_min_dist,
                             match_condition= '>=',)

predictions_isolated_NN = inference(instance=instance,
                                    X=instance.X_train.loc[isolated_NN.index],
                                    y=instance.y_train.loc[isolated_NN.index],
                                    )
print("\nHere is one such 'isolated' record and its nearest neighbors, with our model's predictions:\n")
display(predictions_isolated_NN)

print("\nHere is the nearest neighbor distance distribution for records in the minority class.")
min_distances2 = min_distances.loc[y[y==2].dropna().index]
min_dist2_vc = min_distances2.value_counts()
min_distances2_df = min_distances2.to_frame(name='nn dist')
display(combined_value_counts(df=min_distances2_df,column='nn dist').sort_values(by='nn dist'))


FOLD 0

Value d at index idx means:

- There exists at least one record in df that differs from df.loc[idx] in exactly d features, and
- No other record in df differs from df.loc[idx] in fewer than d features.

Note: Some values may be incorrect as we're using an approximate nearest neighbors approach for efficiency.
      This may occasionally miss the exact closest match in large or high-dimensional datasets.

993546     1
198122     0
847005     1
685049     1
544077     0
          ..
25782      0
1294004    1
1135316    2
209294     1
19868      1
Length: 224703, dtype: int64

Here are the value counts:



,nn dist,count,%
0,0,95374,42.444471
1,1,87866,39.103172
2,2,32862,14.624638
3,3,7428,3.305697
4,4,1073,0.477519
5,5,94,0.041833
6,6,6,0.002670



E.g. there are 6 records in df that differ from every other record in df in at least 6 features.

Here is one such 'isolated' record and its nearest neighbors, with our model's predictions:



,MONTH,HOUR,WKDY_WKND,NUM_VEH,SPD_LIM,ACCDN_TYPE,RD_COND,LIGHT,ZONE,PUB_PRIV_RD,...,HVY_VEH,MTRCYC,BICYC,PED,TNRY_SEV,prob 0,prob 1,prob 2,pred,adj pred
1422120,3,2.0,0,0,NaN,1.0,1.0,3.0,1.0,NaN,...,0,0,0,1,1,0.000249,0.956091,0.043660,1,1
685544,5,5.0,0,0,NaN,1.0,0.0,1.0,1.0,1.0,...,0,0,0,1,1,0.000495,0.896627,0.102878,1,2
835664,4,0.0,0,0,NaN,1.0,0.0,1.0,1.0,1.0,...,0,0,0,1,1,0.000656,0.731196,0.268148,1,2
997725,5,2.0,0,0,NaN,1.0,0.0,3.0,1.0,1.0,...,0,0,0,1,1,0.000447,0.936385,0.063168,1,2
1152010,5,2.0,0,0,NaN,1.0,0.0,3.0,1.0,0.0,...,0,0,0,1,1,0.000586,0.939885,0.059529,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379632,10,1.0,0,0,NaN,1.0,1.0,3.0,1.0,0.0,...,0,0,0,1,1,0.000253,0.946079,0.053668,1,2
211031,7,3.0,0,0,NaN,1.0,0.0,3.0,1.0,1.0,...,0,0,0,1,1,0.000554,0.921387,0.078059,1,2
378237,10,4.0,1,0,NaN,1.0,0.0,0.0,1.0,1.0,...,0,0,0,1,1,0.000931,0.856767,0.142301,1,2
31298,5,NaN,0,1,NaN,0.0,1.0,3.0,2.0,1.0,...,0,0,0,0,0,0.985446,0.014154,0.000400,0,0



Here is the nearest neighbor distance distribution for records in the minority class.


,nn dist,count,%
1,0,534,27.596899
0,1,768,39.689922
2,2,439,22.687339
3,3,160,8.268734
4,4,31,1.602067
5,6,3,0.155039


<a id='conclusion'></a>
## Conclusion
↑↑ [Contents](#contents) ↑ [Retrospective analysis](#analysis)

We started with a dataset of **1,717,407** publicly available police records of traffic accidents in Quebec from **2011 to 2022**. Each accident was categorized into one of four classes. To focus on the presence and severity of injuries, we merged the two "material damage only" categories into one, transforming the problem into a **ternary classification**:

- **Class 0**: Accidents with no injuries
- **Class 1**: Accidents with minor injuries
- **Class 2**: Accidents with severe injuries or fatalities

Focusing on the Montreal region, we narrowed the dataset down to **330,446** records. We performed a stratified train-test split, allocating **85%** (280,879 records) for stratified 5-fold cross-validation and reserving **15%** (49,567 records) for testing. In four of the five folds, the training set comprised 224,703 records and the validation set 56,176 records; the final fold had 224,704 training records and 56,175 validation records.

After removing fields incompatible with our classification objective—such as "number of victims," which directly indicates whether an accident involved injuries—we retained **20 features**. These included:

- **Temporal variables**: Month, weekend/weekday, hour
- **Environmental factors**: Weather, light conditions
- **Road conditions and built-environment characteristics**: Speed limit, one-way or two-way streets, school/residential zones
- **Types of road users or vehicles involved**: Pedestrians, cyclists, motorcyclists, light trucks, heavy vehicles
- **Nature of the accident**: Single vehicle, collision with a fixed object, animal involvement, etc.

We treated three features as **ordinal** (number of vehicles, light conditions, and speed limit) and the remaining 17 as **categorical**. We also noted interdependencies among features; for example, speed limits in school zones cannot exceed 50 km/h when school is in session but may be higher during summer months.

The dataset was **highly imbalanced**:

- **Class 0**: Approximately 79.8% of records
- **Class 1**: Approximately 19.3% of records
- **Class 2**: Only 0.89% of records

Misclassification risks varied by class:

- Misclassifying a minor accident as severe could lead to unnecessary resource allocation.
- Misclassifying a severe accident as minor could have life-threatening consequences.

Therefore, we prioritized both **precision and recall**, but with different emphases across classes:

- For **Class 2**, **recall** was paramount to ensure severe accidents were rarely missed.
- For **Class 0**, **precision** was more important to avoid false positives (more severe accidents being classified as material damage only).
- For **Class 1**, we considered precision and recall equally important.

To evaluate our model's performance, we used the $ F_{\beta} $ score with class-specific beta values:

- $\beta = \frac{1}{2} $ for **Class 0**
- $ \beta = 1 $ for **Class 1**
- $ \beta = 2 $ for **Class 2**

We computed a weighted average of these scores to create an overall evaluation metric, assigning weights of **1**, **4**, and **16** to Classes 0, 1, and 2 respectively, reflecting both their importance and the class imbalance. We denoted this metric as $ F^* $.

We chose the **XGBoost Classifier** due to its ability to automatically explore feature interactions, handle missing values effectively, and its computational efficiency—making it well-suited for capturing complex relationships in our dataset. We used **`mlogloss`** (multiclass logarithmic loss) as the loss function, which penalizes false confidence in misclassifications and encourages well-calibrated probability estimates across classes.

**Additionally, our code and pipeline are designed with flexibility in mind. They allow for easy modifications such as changing the problem to a binary classification task, adding or removing features, or experimenting with different classifiers. This adaptability facilitates rapid prototyping and testing of various modeling approaches to find the most effective solution. However, as we progressively added functions and classes throughout the project, the codebase became somewhat unwieldy and ad hoc. As a result, refactoring is needed to improve its structure and maintainability.**

Our baseline model performed poorly, correctly identifying only about **4 out of 484** severe accidents and fewer than **3,900 out of 10,866** minor accidents in each validation fold, resulting in a low $ F^* $ score of approximately **0.14**. We improved the $ F^* $ score to around **0.32** and identified over **200** Class 2 accidents by adjusting classification thresholds, although these thresholds were somewhat arbitrary. Specifically, we classified an accident as Class 2 if the predicted probability was at least **5%**; if not, as Class 1 if the probability was at least **5%**; and as Class 0 otherwise.

To address class imbalance, we explored **simple oversampling** and **augmented oversampling** using records from outside the Montreal region. Augmented oversampling slightly outperformed simple oversampling but still yielded weaker results than threshold adjustment alone. We also experimented with combining threshold adjustment and oversampling.

We conducted a partial, ad hoc **grid search** to optimize oversampling rates along with hyperparameters like **`max_depth`** (3 and 6) and **`n_estimators`** (100 and 150) for the XGBoost Classifier but observed no improvement in performance.

At this point, we suspected that further hyperparameter tuning would offer little benefit. Since XGBoost inherently captures feature interactions, explicitly adding interaction terms was also unlikely to enhance performance. Evaluating our final model on the test set showed similar results to the validation folds, leading us to focus on understanding the reasons behind the model's poor performance.

We observed that the model **underfit** the data, performing as poorly on the training set as on the validation set. We posited that this underfitting resulted from a fundamental lack of **discriminative power** in the features. **Mutual information scores were uniformly low**, and we found instances where identical or very similar feature combinations appeared multiple times but belonged to different classes. Clusters of similar feature combinations, differing in only one or two seemingly unimportant features, were distributed across multiple classes. This lack of consistent class separation hindered the model's ability to learn meaningful patterns.

Our conclusion is that reliably classifying an accident’s severity likely requires additional types of features. We speculate that **human factors**—such as driver and victim age, driver history, behavior prior to the accident, DUI status, and similar variables—may have a stronger influence on severity levels. Emphasizing these human factors could improve classification performance. We also recommend exploring ways to make more detailed information publicly available while respecting privacy laws. Finally, we propose our $ F^* $ score (or a similar metric) as a potential benchmark for future analyses of traffic accident severity.